<a href="https://colab.research.google.com/github/koba341/AutoML/blob/main/AdvancedRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# General Settings
auto-sklearn hat Kompatiblitätsschwierigkeiten mit der scipy-Bibliothek. \\ Bisheriger Workaround: Zwischen scipy Version 1.4.0 und 1.7.3 hin und her up- und downgraden und dann Laufzeit neustarten bis es keine Fehlermeldung mehr gibt.

In [ ]:
!pip install auto-sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 7.7 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 22.3 MB 11.9 MB/s 
     |████████████████████████████████| 1.1 MB 11.2 MB/s 
     |████████████████████████████████| 837 kB 49.6 MB/s 
     |████████████████████████████████| 4.3 MB 44.4 MB/s 
     |████████████████████████████████| 4.4 MB 43.5 MB/s 
     |████████████████████████████████| 260 kB 42.8 MB/s 
     |████████████████████████████████| 596 kB 32.9 MB/s 
     |████████████████████████████████| 140 kB 15.2 MB/s 
     |████████████████████████████████| 46 kB 1.5 MB/s 
  Created wheel for auto-sklearn: filename=auto_sklearn-0.14.7-py3-none-any.whl size=6602873 sha256=1360b7ae9f86149cebf1548a672744f5f82bf65a3d8a22f13e36ebc931055633
  Stored in directory: /root/.cache/pip/wheels/ba/43/5c/2fbe6fd19e3af314cbc4aa808378068d8ddd6792064f

In [ ]:
#!pip install scipy==1.7.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scipy-1.7.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (38.1 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.1.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import warnings as wrn
wrn.filterwarnings('ignore', category = DeprecationWarning) 
wrn.filterwarnings('ignore', category = FutureWarning) 
wrn.filterwarnings('ignore', category = UserWarning) 

'''Import basic modules.'''
import pandas as pd
import numpy as np
from scipy import stats

'''Customize visualization
Seaborn and matplotlib visualization.'''
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('bmh')                    
sns.set_style({'axes.grid':False}) 

'''Plotly visualization .'''
import plotly.offline as py
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True) # Required to use plotly offline in jupyter notebook

'''Display markdown formatted output like bold, italic bold etc.'''
from IPython.display import Markdown
def bold(string):
    display(Markdown(string))

import plotly.io as pio
pio.renderers.default='colab'

import scipy
import autosklearn
import autosklearn.regression

In [ ]:
'''Function to plot bar chart'''
def bar_plot(x, y, title, yaxis, c_scale):
    trace = go.Bar(
    x = x,
    y = y,
    marker = dict(color = y, colorscale = c_scale))
    layout = go.Layout(hovermode= 'closest', title = title, yaxis = dict(title = yaxis))
    fig = go.Figure(data = [trace], layout = layout)
    return iplot(fig)

'''Function to plot scatter plot'''
def scatter_plot(x, y, title, xaxis, yaxis, size, c_scale):
    trace = go.Scatter(
    x = x,
    y = y,
    mode = 'markers',
    marker = dict(color = y, size = size, showscale = True, colorscale = c_scale))
    layout = go.Layout(hovermode= 'closest', title = title, xaxis = dict(title = xaxis), yaxis = dict(title = yaxis))
    fig = go.Figure(data = [trace], layout = layout)
    return iplot(fig)    
    
'''Function to plot histogram'''
def plot_histogram(x, title, yaxis, color):
    trace = go.Histogram(x = x,
                        marker = dict(color = color))
    layout = go.Layout(hovermode= 'closest', title = title, yaxis = dict(title = yaxis))
    fig = go.Figure(data = [trace], layout = layout)
    return iplot(fig)

In [ ]:
from sklearn.model_selection import train_test_split
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AutoML/slump_test.csv")
X = data.iloc[:, 1:10].to_numpy()
y = data.iloc[:, 10].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.90, random_state=1) # create training and test dataset. # test set rausnehmen.
data_cols = data.columns.copy().tolist()
data_cols.remove('No')
data_cols.remove('Compressive Strength (28-day)(Mpa)')

In [ ]:
'''Set a seed for reproducibility'''
seed = 1

'''Initialize all the regression models object we are interested in.'''
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

'''We are interested in the following 14 regression models.
All initialized with default parameters except random_state and n_jobs.'''
linear = LinearRegression(n_jobs = -1)
lasso = Lasso(random_state = seed)
ridge = Ridge(random_state = seed)
kr = KernelRidge()
elnt = ElasticNet(random_state = seed)
dt = DecisionTreeRegressor(random_state = seed)
svm = SVR()
knn = KNeighborsRegressor(n_jobs = -1)
rf =  RandomForestRegressor(n_jobs = -1, random_state = seed)
et = ExtraTreesRegressor(n_jobs = -1, random_state = seed)
ab = AdaBoostRegressor(random_state = seed)
gb = GradientBoostingRegressor(random_state = seed)
xgb = XGBRegressor(random_state = seed, n_jobs = -1)
gpr = GaussianProcessRegressor(random_state= seed)
mlp = MLPRegressor(random_state=seed)

In [ ]:
def train_test_split_score(model):
    from sklearn.metrics import mean_squared_error    
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    mse = mean_squared_error(prediction, y_test)
    rmse = np.sqrt(mse)
    return rmse

# Train and Predict with default Hyperparameters.

In [ ]:
'''Training accuracy of our regression models. By default score method returns coefficient of determination (r_squared).'''
def train_r2(model):
    model.fit(X_train, y_train)
    return model.score(X_train, y_train)

'''Calculate and plot the training accuracy.'''
models = [linear, lasso, ridge, kr, elnt, dt, svm, knn, rf, et, ab, gb, xgb, gpr, mlp]
training_score = []
for model in models:
    training_score.append(train_r2(model))
    
'''Plot dataframe of training accuracy.'''
train_score = pd.DataFrame(data = training_score, columns = ['Training_R2'])
train_score.index = ['LR', 'LSO', 'RIDGE', 'KR', 'ELNT', 'DT', 'SVM', 'KNN', 'RF', 'ET', 'AB', 'GB', 'XGB', 'GPR', 'MLP']
train_score = (train_score*100).round(4)
scatter_plot(train_score.index, train_score['Training_R2'], 'Training Score (R_Squared)', 'Models','% Training Score', 30, 'Rainbow')

import os  

os.makedirs('/content/drive/MyDrive/Colab Notebooks/AutoML', exist_ok=True)  

train_score.to_csv('/content/drive/MyDrive/Colab Notebooks/AutoML/train_score_defaultParams.csv')  

[08:52:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
models = [linear, lasso, ridge, kr, elnt, dt, svm, knn, rf, et, ab, gb, xgb, gpr, mlp]
train_test_split_rmse = []
for model in models:
    train_test_split_rmse.append(train_test_split_score(model))

'''Plot data frame of train test rmse'''
train_test_score = pd.DataFrame(data = train_test_split_rmse, columns = ['Train_Test_RMSE'])
train_test_score.index = ['LR', 'LSO', 'RIDGE', 'KR', 'ELNT', 'DT', 'SVM', 'KNN', 'RF', 'ET', 'AB', 'GB', 'XGB', 'GPR', 'MLP']
train_test_score = train_test_score.round(5)
x = train_test_score.index
y = train_test_score['Train_Test_RMSE']
title = "Models' Test Score (RMSE) on Holdout(10%) Set"
scatter_plot(x, y, title, 'Models','RMSE', 30, 'RdBu')


import os  

os.makedirs('/content/drive/MyDrive/Colab Notebooks/AutoML', exist_ok=True)  

train_test_score.to_csv('/content/drive/MyDrive/Colab Notebooks/AutoML/train_test_score_defaultParams.csv')  

[10:30:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
'''Function to compute cross validation scores.'''
def cross_validate(model):
    from sklearn.model_selection import cross_val_score
    neg_x_val_score = cross_val_score(model, X_train, y_train, cv = 10, n_jobs = -1, scoring = 'neg_mean_squared_error')
    x_val_score = np.round(np.sqrt(-1*neg_x_val_score), 5)
    return x_val_score.mean()

'''Calculate cross validation score of differnt models and plot them.'''
models = [lasso, ridge, kr, elnt, dt, svm, knn, rf, et, ab, gb, xgb, gpr, mlp]
cross_val_scores = []
for model in models:
    cross_val_scores.append(cross_validate(model))

'''Plot data frame of cross validation scores.'''
x_val_score = pd.DataFrame(data = cross_val_scores, columns = ['Cross Validation Scores (RMSE)'])
x_val_score.index = ['LSO', 'RIDGE', 'KR', 'ELNT', 'DT', 'SVM', 'KNN', 'RF', 'ET', 'AB', 'GB', 'XGB', 'GPR', 'MLP']
x_val_score = x_val_score.round(5)
x = x_val_score.index
y = x_val_score['Cross Validation Scores (RMSE)']
title = "Models' 10-fold Cross Validation Scores (RMSE)"
scatter_plot(x, y, title, 'Models','RMSE', 30, 'Viridis')


import os  

os.makedirs('/content/drive/MyDrive/Colab Notebooks/AutoML', exist_ok=True)  

train_score.to_csv('/content/drive/MyDrive/Colab Notebooks/AutoML/10fCV_defaultParams.csv')  

# General AutoML Settings


In [ ]:
time_left_for_this_task=180
per_run_time_limit=20

# LinearRegression Comparison

No AutoML needed because it has no Hyperparameters.

# Optimize Lasso with AutoML

The optimization objective for Lasso is:

$(1 / (2 \cdot n_{samples})) \cdot ||y - Xw||^2_2 + \alpha \cdot ||w||_1$


Hyperparameters needed to be tuned: $\alpha$

In [ ]:
# First Step: Extend autosklearn with Lasso.
from ConfigSpace.configuration_space import ConfigurationSpace
from ConfigSpace.hyperparameters import UniformFloatHyperparameter, \
    UniformIntegerHyperparameter, CategoricalHyperparameter
from ConfigSpace.conditions import EqualsCondition

import sklearn.metrics
import autosklearn.regression
import autosklearn.pipeline.components.regression
from autosklearn.pipeline.components.base import AutoSklearnRegressionAlgorithm
from autosklearn.pipeline.constants import SPARSE, DENSE, \
    SIGNED_DATA, UNSIGNED_DATA, PREDICTIONS

class LassoRegression(AutoSklearnRegressionAlgorithm):

    def __init__(self, alpha, normalize, random_state=None):
        self.alpha = alpha
        self.normalize = normalize
        self.random_state = random_state
        self.estimator = None

    def fit(self, X, y):
        self.alpha = float(self.alpha)

        from sklearn.linear_model import Lasso
        self.estimator = Lasso(
            alpha=self.alpha, 
            normalize=self.normalize
        )
        self.estimator.fit(X, y)
        return self

    def predict(self, X):
        if self.estimator is None:
            raise NotImplementedError
        return self.estimator.predict(X)

    @staticmethod
    def get_properties(dataset_properties=None):
        return {
            'shortname': 'L1',
            'name': 'Lasso Regression',
            'handles_regression': True,
            'handles_classification': False,
            'handles_multiclass': False,
            'handles_multilabel': False,
            'handles_multioutput': False,
            'is_deterministic': True,
            'input': (SPARSE, DENSE, UNSIGNED_DATA, SIGNED_DATA),
            'output': (PREDICTIONS,)
        }

    @staticmethod
    def get_hyperparameter_search_space(dataset_properties=None):
        cs = ConfigurationSpace()
        alpha = UniformFloatHyperparameter(
            name='alpha', lower=10 ** -3, upper=10 ** 2, log=True, default_value=0.1
        )
        normalize = CategoricalHyperparameter(
            name='normalize',
            choices=['True', 'False'],
            default_value='True'
        )
        cs.add_hyperparameters([alpha, normalize])
        return cs


# Add Lasso component to auto-sklearn.
autosklearn.pipeline.components.regression.add_regressor(LassoRegression)
cs = LassoRegression.get_hyperparameter_search_space()
print(cs)

Configuration space object:
  Hyperparameters:
    alpha, Type: UniformFloat, Range: [0.001, 100.0], Default: 0.1, on log-scale
    normalize, Type: Categorical, Choices: {True, False}, Default: True



In [ ]:
# Second Step: Optimize alpha Hyperparameter with automl.

reg = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=time_left_for_this_task,
    per_run_time_limit=per_run_time_limit,
    include={
        'regressor': ['LassoRegression'],
        'feature_preprocessor': ['no_preprocessing']
    },
    
    ensemble_size=1
)
reg.fit(X_train, y_train)

[WARNING] [2022-06-29 08:55:23,993:Client-AutoMLSMBO(1)::3622a388-f789-11ec-8204-0242ac1c0002] Configuration 197 not found
[WARNING] [2022-06-29 08:55:23,993:Client-AutoMLSMBO(1)::3622a388-f789-11ec-8204-0242ac1c0002] Configuration 235 not found
[WARNING] [2022-06-29 08:55:23,993:Client-AutoMLSMBO(1)::3622a388-f789-11ec-8204-0242ac1c0002] Configuration 270 not found
[WARNING] [2022-06-29 08:55:23,993:Client-AutoMLSMBO(1)::3622a388-f789-11ec-8204-0242ac1c0002] Configuration 163 not found
[WARNING] [2022-06-29 08:55:23,993:Client-AutoMLSMBO(1)::3622a388-f789-11ec-8204-0242ac1c0002] Configuration 67 not found
[WARNING] [2022-06-29 08:55:23,993:Client-AutoMLSMBO(1)::3622a388-f789-11ec-8204-0242ac1c0002] Configuration 34 not found
[WARNING] [2022-06-29 08:55:23,993:Client-AutoMLSMBO(1)::3622a388-f789-11ec-8204-0242ac1c0002] Configuration 86 not found
[WARNING] [2022-06-29 08:55:23,993:Client-AutoMLSMBO(1)::3622a388-f789-11ec-8204-0242ac1c0002] Configuration 240 not found
[WARNING] [2022-06-

AutoSklearnRegressor(ensemble_size=1,
                     include={'feature_preprocessor': ['no_preprocessing'],
                              'regressor': ['LassoRegression']},
                     per_run_time_limit=20, time_left_for_this_task=180)

In [ ]:
# Third Step: Save best params and best score.
model_dict = reg.show_models()
key = list(model_dict.keys())[0]
lasso_best_param = model_dict[key]['sklearn_regressor'].alpha
lasso_best_score = model_dict[key]['cost']

print('Best Parameter: {}. Best Score: {}'.format(lasso_best_param, lasso_best_score))
lassoParams = model_dict[key]['sklearn_regressor'].get_params()

Best Parameter: 0.013044763053548238. Best Score: 0.13208339117685952


# Optimize Ridge

Linear least squares with l2 regularization.

Minimizes the objective function:

$||y - Xw||^2_2 + \alpha \cdot ||w||^2_2$

We want to optimize alpha.

In [ ]:
# First Step: Extend autosklearn with Ridge.
from ConfigSpace.configuration_space import ConfigurationSpace
from ConfigSpace.hyperparameters import UniformFloatHyperparameter, \
    UniformIntegerHyperparameter, CategoricalHyperparameter
from ConfigSpace.conditions import EqualsCondition

import sklearn.metrics
import autosklearn.regression
import autosklearn.pipeline.components.regression
from autosklearn.pipeline.components.base import AutoSklearnRegressionAlgorithm
from autosklearn.pipeline.constants import SPARSE, DENSE, \
    SIGNED_DATA, UNSIGNED_DATA, PREDICTIONS

class RidgeRegression(AutoSklearnRegressionAlgorithm):

    def __init__(self, alpha, normalize, random_state=None):
        self.alpha = alpha
        self.normalize = normalize
        self.random_state = random_state
        self.estimator = None

    def fit(self, X, y):
        self.alpha = float(self.alpha)

        from sklearn.linear_model import Ridge
        self.estimator = Ridge(
            alpha=self.alpha, 
            normalize=self.normalize
        )
        self.estimator.fit(X, y)
        return self

    def predict(self, X):
        if self.estimator is None:
            raise NotImplementedError
        return self.estimator.predict(X)

    @staticmethod
    def get_properties(dataset_properties=None):
        return {
            'shortname': 'L2',
            'name': 'Ridge Regression',
            'handles_regression': True,
            'handles_classification': False,
            'handles_multiclass': False,
            'handles_multilabel': False,
            'handles_multioutput': False,
            'is_deterministic': True,
            'input': (SPARSE, DENSE, UNSIGNED_DATA, SIGNED_DATA),
            'output': (PREDICTIONS,)
        }

    @staticmethod
    def get_hyperparameter_search_space(dataset_properties=None):
        cs = ConfigurationSpace()
        alpha = UniformFloatHyperparameter(
            name='alpha', lower=10 ** -3, upper=10 ** 2, log=True, default_value=0.1
        )
        normalize = CategoricalHyperparameter(
            name='normalize',
            choices=['True', 'False'],
            default_value='True'
        )
        cs.add_hyperparameters([alpha, normalize])
        return cs


# Add Ridge component to auto-sklearn.
autosklearn.pipeline.components.regression.add_regressor(RidgeRegression)
cs = RidgeRegression.get_hyperparameter_search_space()
print(cs)

Configuration space object:
  Hyperparameters:
    alpha, Type: UniformFloat, Range: [0.001, 100.0], Default: 0.1, on log-scale
    normalize, Type: Categorical, Choices: {True, False}, Default: True



In [ ]:
# Second Step: Optimize alpha Hyperparameter with automl.

reg = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=time_left_for_this_task,
    per_run_time_limit=per_run_time_limit,
    include={
        'regressor': ['RidgeRegression'],
        'feature_preprocessor': ['no_preprocessing']
    },
    ensemble_size=1
)
reg.fit(X_train, y_train)

[WARNING] [2022-06-29 08:58:17,949:Client-AutoMLSMBO(1)::9da8ceac-f789-11ec-8204-0242ac1c0002] Configuration 197 not found
[WARNING] [2022-06-29 08:58:17,949:Client-AutoMLSMBO(1)::9da8ceac-f789-11ec-8204-0242ac1c0002] Configuration 235 not found
[WARNING] [2022-06-29 08:58:17,949:Client-AutoMLSMBO(1)::9da8ceac-f789-11ec-8204-0242ac1c0002] Configuration 270 not found
[WARNING] [2022-06-29 08:58:17,949:Client-AutoMLSMBO(1)::9da8ceac-f789-11ec-8204-0242ac1c0002] Configuration 163 not found
[WARNING] [2022-06-29 08:58:17,949:Client-AutoMLSMBO(1)::9da8ceac-f789-11ec-8204-0242ac1c0002] Configuration 67 not found
[WARNING] [2022-06-29 08:58:17,949:Client-AutoMLSMBO(1)::9da8ceac-f789-11ec-8204-0242ac1c0002] Configuration 34 not found
[WARNING] [2022-06-29 08:58:17,949:Client-AutoMLSMBO(1)::9da8ceac-f789-11ec-8204-0242ac1c0002] Configuration 86 not found
[WARNING] [2022-06-29 08:58:17,949:Client-AutoMLSMBO(1)::9da8ceac-f789-11ec-8204-0242ac1c0002] Configuration 240 not found
[WARNING] [2022-06-

AutoSklearnRegressor(ensemble_size=1,
                     include={'feature_preprocessor': ['no_preprocessing'],
                              'regressor': ['RidgeRegression']},
                     per_run_time_limit=20, time_left_for_this_task=180)

In [ ]:
# Third Step: Save best params and best score.
model_dict = reg.show_models()
key = list(model_dict.keys())[0]
ridge_best_param = model_dict[key]['sklearn_regressor'].alpha
ridge_best_score = model_dict[key]['cost']

print('Best Parameter: {}. Best Score: {}'.format(ridge_best_param, ridge_best_score))
ridgeParams = model_dict[key]['sklearn_regressor'].get_params()

Best Parameter: 0.03697684148420542. Best Score: 0.12789937230971504


# Optimize ElasticNet

Linear regression with combined L1 and L2 priors as regularizer.

Minimizes the objective function:

$1 / (2 \cdot n_{samples}) \cdot ||y - Xw||^2_2
+ \alpha \cdot l1\_ratio \cdot ||w||_1
+ 0.5 \cdot \alpha \cdot (1 - l1\_ratio) * ||w||^2_2$

We want to optimize the Hyperparameters alpha and l1_ratio.


In [ ]:
# First Step: Extend autosklearn with ElasticNet.
from ConfigSpace.configuration_space import ConfigurationSpace
from ConfigSpace.hyperparameters import UniformFloatHyperparameter, \
    UniformIntegerHyperparameter, CategoricalHyperparameter
from ConfigSpace.conditions import EqualsCondition

import sklearn.metrics
import autosklearn.regression
import autosklearn.pipeline.components.regression
from autosklearn.pipeline.components.base import AutoSklearnRegressionAlgorithm
from autosklearn.pipeline.constants import SPARSE, DENSE, \
    SIGNED_DATA, UNSIGNED_DATA, PREDICTIONS

class ElasticNetRegression(AutoSklearnRegressionAlgorithm):

    def __init__(self, alpha, l1_ratio, normalize, random_state=None):
        self.alpha = alpha
        self.normalize = normalize
        self.l1_ratio = l1_ratio
        self.random_state = random_state
        self.estimator = None

    def fit(self, X, y):
        self.alpha = float(self.alpha)
        self.l1_ratio = float(self.l1_ratio)

        from sklearn.linear_model import ElasticNet
        self.estimator = ElasticNet(
            alpha=self.alpha,
            l1_ratio=self.l1_ratio, 
            normalize=self.normalize
        )
        self.estimator.fit(X, y)
        return self

    def predict(self, X):
        if self.estimator is None:
            raise NotImplementedError
        return self.estimator.predict(X)

    @staticmethod
    def get_properties(dataset_properties=None):
        return {
            'shortname': 'EN',
            'name': 'Elastic Net Regression',
            'handles_regression': True,
            'handles_classification': False,
            'handles_multiclass': False,
            'handles_multilabel': False,
            'handles_multioutput': False,
            'is_deterministic': True,
            'input': (SPARSE, DENSE, UNSIGNED_DATA, SIGNED_DATA),
            'output': (PREDICTIONS,)
        }

    @staticmethod
    def get_hyperparameter_search_space(dataset_properties=None):
        cs = ConfigurationSpace()
        alpha = UniformFloatHyperparameter(
            name='alpha', lower=10 ** -3, upper=10 ** 2, log=True, default_value=0.1
        )
        normalize = CategoricalHyperparameter(
            name='normalize',
            choices=['True', 'False'],
            default_value='True'
        )
        l1_ratio = UniformFloatHyperparameter(
            name='l1_ratio', lower=10 ** -3, upper=10 ** 0, log=True, default_value=0.1
        )
        cs.add_hyperparameters([alpha, l1_ratio, normalize])
        return cs


# Add Elastic Net component to auto-sklearn.
autosklearn.pipeline.components.regression.add_regressor(ElasticNetRegression)
cs = ElasticNetRegression.get_hyperparameter_search_space()
print(cs)

Configuration space object:
  Hyperparameters:
    alpha, Type: UniformFloat, Range: [0.001, 100.0], Default: 0.1, on log-scale
    l1_ratio, Type: UniformFloat, Range: [0.001, 1.0], Default: 0.1, on log-scale
    normalize, Type: Categorical, Choices: {True, False}, Default: True



In [ ]:
# Second Step: Optimize Hyperparameter with automl.

reg = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=time_left_for_this_task,
    per_run_time_limit=per_run_time_limit,
    include={
        'regressor': ['ElasticNetRegression'],
        'feature_preprocessor': ['no_preprocessing']
    },
    ensemble_size=1
)
reg.fit(X_train, y_train)

[WARNING] [2022-06-29 09:01:11,792:Client-AutoMLSMBO(1)::0572a7fe-f78a-11ec-8204-0242ac1c0002] Configuration 197 not found
[WARNING] [2022-06-29 09:01:11,792:Client-AutoMLSMBO(1)::0572a7fe-f78a-11ec-8204-0242ac1c0002] Configuration 235 not found
[WARNING] [2022-06-29 09:01:11,792:Client-AutoMLSMBO(1)::0572a7fe-f78a-11ec-8204-0242ac1c0002] Configuration 270 not found
[WARNING] [2022-06-29 09:01:11,792:Client-AutoMLSMBO(1)::0572a7fe-f78a-11ec-8204-0242ac1c0002] Configuration 163 not found
[WARNING] [2022-06-29 09:01:11,792:Client-AutoMLSMBO(1)::0572a7fe-f78a-11ec-8204-0242ac1c0002] Configuration 67 not found
[WARNING] [2022-06-29 09:01:11,792:Client-AutoMLSMBO(1)::0572a7fe-f78a-11ec-8204-0242ac1c0002] Configuration 34 not found
[WARNING] [2022-06-29 09:01:11,793:Client-AutoMLSMBO(1)::0572a7fe-f78a-11ec-8204-0242ac1c0002] Configuration 86 not found
[WARNING] [2022-06-29 09:01:11,793:Client-AutoMLSMBO(1)::0572a7fe-f78a-11ec-8204-0242ac1c0002] Configuration 240 not found
[WARNING] [2022-06-

AutoSklearnRegressor(ensemble_size=1,
                     include={'feature_preprocessor': ['no_preprocessing'],
                              'regressor': ['ElasticNetRegression']},
                     per_run_time_limit=20, time_left_for_this_task=180)

In [ ]:
# Third Step: Save best params and best score.
model_dict = reg.show_models()
key = list(model_dict.keys())[0]
en_best_params = [model_dict[key]['sklearn_regressor'].alpha, model_dict[key]['sklearn_regressor'].l1_ratio]
en_best_score = model_dict[key]['cost']

print('Best Parameter for alpha: {}. Best Parameter for l1_ratio: {}. Best Score: {}'.format(en_best_params[0], en_best_params[0], en_best_score))
enParams = model_dict[key]['sklearn_regressor'].get_params()

Best Parameter for alpha: 0.0010466106277878216. Best Parameter for l1_ratio: 0.0010466106277878216. Best Score: 0.12820084141411858


# Optimize Support Vector Machine

Hyperparameters: \\
C: [0.03125, 3278] \\
epsilon: [0.001, 1] \\
gamma (Kernel Coefficient for rbf, poly and sigmoid): [3.0517578125e-05, 8] \\
degree (Degree of the polynomial kernel function (‘poly’)):
Kernel: 'linear', 'poly', 'rbf', 'sigmoid' \\

In [ ]:
# First Step: Extend autosklearn

from ConfigSpace.configuration_space import ConfigurationSpace
from ConfigSpace.conditions import InCondition, EqualsCondition
from ConfigSpace.hyperparameters import UniformFloatHyperparameter, \
    UniformIntegerHyperparameter, CategoricalHyperparameter, \
    UnParametrizedHyperparameter
from autosklearn.pipeline.components.base import AutoSklearnRegressionAlgorithm
from autosklearn.pipeline.constants import DENSE, UNSIGNED_DATA, PREDICTIONS, SPARSE
from autosklearn.util.common import check_for_bool, check_none


class LibSVM_SVR(AutoSklearnRegressionAlgorithm):
    def __init__(self, kernel, C, epsilon, gamma=0.1,
                 degree=3, coef0=0.0, random_state=None):
        self.kernel = kernel
        self.C = C
        self.epsilon = epsilon
        self.degree = degree
        self.gamma = gamma
        self.coef0 = coef0
        self.random_state = random_state
        self.estimator = None

    def fit(self, X, y):
        import sklearn.svm

        # Calculate the size of the kernel cache (in MB) for sklearn's LibSVM. The cache size is
        # calculated as 2/3 of the available memory (which is calculated as the memory limit minus
        # the used memory)
        try:
            # Retrieve memory limits imposed on the process
            soft, hard = resource.getrlimit(resource.RLIMIT_AS)

            if soft > 0:
                # Convert limit to units of megabytes
                soft /= 1024 * 1024

                # Retrieve memory used by this process
                maxrss = resource.getrusage(resource.RUSAGE_SELF)[2] / 1024

                # In MacOS, the MaxRSS output of resource.getrusage in bytes; on other platforms,
                # it's in kilobytes
                if sys.platform == 'darwin':
                    maxrss = maxrss / 1024

                cache_size = (soft - maxrss) / 1.5

                if cache_size < 0:
                    cache_size = 200
            else:
                cache_size = 200
        except Exception:
            cache_size = 200

        self.C = float(self.C)
        self.epsilon = float(self.epsilon)
        self.degree = int(self.degree)
        self.gamma = float(self.gamma)
        if check_none(self.coef0):
            self.coef0 = 0.0
        else:
            self.coef0 = float(self.coef0)

        self.estimator = sklearn.svm.SVR(
            kernel=self.kernel,
            C=self.C,
            epsilon=self.epsilon,
            degree=self.degree,
            gamma=self.gamma,
            coef0=self.coef0
        )
        self.scaler = sklearn.preprocessing.StandardScaler(copy=True)

        # Convert y to be at least 2d for the scaler
        # [1,1,1] -> [[1], [1], [1]]
        if y.ndim == 1:
            y = y.reshape((-1, 1))

        y_scaled = self.scaler.fit_transform(y)

        # Flatten: [[0], [0], [0]] -> [0, 0, 0]
        if y_scaled.ndim == 2 and y_scaled.shape[1] == 1:
            y_scaled = y_scaled.flatten()

        self.estimator.fit(X, y_scaled)

        return self

    def predict(self, X):
        if self.estimator is None:
            raise NotImplementedError
        if self.scaler is None:
            raise NotImplementedError
        y_pred = self.estimator.predict(X)

        inverse = self.scaler.inverse_transform(y_pred)

        # Flatten: [[0], [0], [0]] -> [0, 0, 0]
        if inverse.ndim == 2 and inverse.shape[1] == 1:
            inverse = inverse.flatten()

        return inverse

    @staticmethod
    def get_properties(dataset_properties=None):
        return {'shortname': 'SVR',
                'name': 'Support Vector Regression',
                'handles_regression': True,
                'handles_classification': False,
                'handles_multiclass': False,
                'handles_multilabel': False,
                'handles_multioutput': False,
                'prefers_data_normalized': True,
                'is_deterministic': True,
                'input': (SPARSE, DENSE, UNSIGNED_DATA),
                'output': (PREDICTIONS,)}

    @staticmethod
    def get_hyperparameter_search_space(dataset_properties=None):
        C = UniformFloatHyperparameter(
            name="C", lower=0.03125, upper=32768, log=True, default_value=1.0)
        # Random Guess
        epsilon = UniformFloatHyperparameter(name="epsilon", lower=0.001,
                                             upper=1, default_value=0.1,
                                             log=True)

        kernel = CategoricalHyperparameter(
            name="kernel", choices=['linear', 'poly', 'rbf', 'sigmoid'],
            default_value="rbf")
        degree = UniformIntegerHyperparameter(
            name="degree", lower=2, upper=5, default_value=3)

        gamma = UniformFloatHyperparameter(
            name="gamma", lower=3.0517578125e-05, upper=8, log=True, default_value=0.1)

        # TODO this is totally ad-hoc
        coef0 = UniformFloatHyperparameter(
            name="coef0", lower=-1, upper=1, default_value=0)

        cs = ConfigurationSpace()
        cs.add_hyperparameters([C, kernel, degree, gamma, coef0, epsilon])

        degree_depends_on_poly = EqualsCondition(degree, kernel, "poly")
        gamma_depends_on_kernel = InCondition(child=gamma, parent=kernel,
                                              values=('poly', 'rbf'))
        coef0_depends_on_kernel = InCondition(child=coef0, parent=kernel,
                                              values=('poly', 'sigmoid'))
        cs.add_conditions([degree_depends_on_poly, gamma_depends_on_kernel,
                           coef0_depends_on_kernel])

        return cs

In [ ]:
# Second Step: Optimize Hyperparameter with automl.

reg = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=time_left_for_this_task,
    per_run_time_limit=per_run_time_limit,
    include={
        'regressor': ['libsvm_svr'],
        'feature_preprocessor': ['no_preprocessing']
    },
    ensemble_size=1
)
reg.fit(X_train, y_train)

[WARNING] [2022-06-29 09:04:05,803:Client-AutoMLSMBO(1)::6cedb518-f78a-11ec-8204-0242ac1c0002] Configuration 197 not found
[WARNING] [2022-06-29 09:04:05,803:Client-AutoMLSMBO(1)::6cedb518-f78a-11ec-8204-0242ac1c0002] Configuration 235 not found
[WARNING] [2022-06-29 09:04:05,803:Client-AutoMLSMBO(1)::6cedb518-f78a-11ec-8204-0242ac1c0002] Configuration 270 not found
[WARNING] [2022-06-29 09:04:05,803:Client-AutoMLSMBO(1)::6cedb518-f78a-11ec-8204-0242ac1c0002] Configuration 163 not found
[WARNING] [2022-06-29 09:04:05,803:Client-AutoMLSMBO(1)::6cedb518-f78a-11ec-8204-0242ac1c0002] Configuration 67 not found
[WARNING] [2022-06-29 09:04:05,803:Client-AutoMLSMBO(1)::6cedb518-f78a-11ec-8204-0242ac1c0002] Configuration 34 not found
[WARNING] [2022-06-29 09:04:05,803:Client-AutoMLSMBO(1)::6cedb518-f78a-11ec-8204-0242ac1c0002] Configuration 86 not found
[WARNING] [2022-06-29 09:04:05,803:Client-AutoMLSMBO(1)::6cedb518-f78a-11ec-8204-0242ac1c0002] Configuration 240 not found
[WARNING] [2022-06-

AutoSklearnRegressor(ensemble_size=1,
                     include={'feature_preprocessor': ['no_preprocessing'],
                              'regressor': ['libsvm_svr']},
                     per_run_time_limit=20, time_left_for_this_task=180)

In [ ]:
# Third Step: Save best params and best score.
model_dict = reg.show_models()
key = list(model_dict.keys())[0]
print(model_dict)

svr_best_params = [model_dict[key]['sklearn_regressor'].kernel, model_dict[key]['sklearn_regressor'].C, model_dict[key]['sklearn_regressor'].epsilon]
svr_best_score = model_dict[key]['cost']

print('Best Parameter for Kernel: {}. Best Parameter for C: {}. Best Parameter for epsilon: {}. Best Score: {}'.format(svr_best_params[0], svr_best_params[1], svr_best_params[2], svr_best_score))
svrParams = model_dict[key]['sklearn_regressor'].get_params()

{25: {'model_id': 25, 'rank': 1, 'cost': 0.02685871308620924, 'ensemble_weight': 1.0, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f76e5c9db50>, 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f76e5dc6750>, 'regressor': <autosklearn.pipeline.components.regression.RegressorChoice object at 0x7f76e5dc63d0>, 'sklearn_regressor': SVR(C=158.05973971461478, cache_size=1882.7447916666667,
    epsilon=0.1297689847703033, gamma=0.025392716718597785, shrinking=False,
    tol=0.003256555639296632, verbose=0)}}
Best Parameter for Kernel: rbf. Best Parameter for C: 158.05973971461478. Best Parameter for epsilon: 0.1297689847703033. Best Score: 0.02685871308620924


# Optimize DecisionTreeRegressor

Hyperparameters: criterion, max_features, max_depth_factor,
                                min_samples_split, min_samples_leaf,
                                min_weight_fraction_leaf, max_leaf_nodes,
                                min_impurity_decrease

In [ ]:
# First Step: Extend autosklearn

from ConfigSpace.configuration_space import ConfigurationSpace
from ConfigSpace.hyperparameters import UniformFloatHyperparameter, \
    UniformIntegerHyperparameter, CategoricalHyperparameter, \
    UnParametrizedHyperparameter, Constant

from autosklearn.pipeline.components.base import \
    AutoSklearnRegressionAlgorithm
from autosklearn.pipeline.constants import DENSE, UNSIGNED_DATA, PREDICTIONS, SPARSE
from autosklearn.util.common import check_none


class DecisionTree(AutoSklearnRegressionAlgorithm):
    def __init__(self, criterion, max_features, max_depth_factor,
                 min_samples_split, min_samples_leaf, min_weight_fraction_leaf,
                 max_leaf_nodes, min_impurity_decrease, random_state=None):
        self.criterion = criterion
        self.max_features = max_features
        self.max_depth_factor = max_depth_factor
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_leaf_nodes = max_leaf_nodes
        self.min_weight_fraction_leaf = min_weight_fraction_leaf
        self.min_impurity_decrease = min_impurity_decrease
        self.random_state = random_state
        self.estimator = None

    def fit(self, X, y, sample_weight=None):
        from sklearn.tree import DecisionTreeRegressor

        self.max_features = float(self.max_features)
        if check_none(self.max_depth_factor):
            max_depth_factor = self.max_depth_factor = None
        else:
            num_features = X.shape[1]
            self.max_depth_factor = int(self.max_depth_factor)
            max_depth_factor = max(
                1,
                int(np.round(self.max_depth_factor * num_features, 0)))
        self.min_samples_split = int(self.min_samples_split)
        self.min_samples_leaf = int(self.min_samples_leaf)
        if check_none(self.max_leaf_nodes):
            self.max_leaf_nodes = None
        else:
            self.max_leaf_nodes = int(self.max_leaf_nodes)
        self.min_weight_fraction_leaf = float(self.min_weight_fraction_leaf)
        self.min_impurity_decrease = float(self.min_impurity_decrease)

        self.estimator = DecisionTreeRegressor(
            criterion=self.criterion,
            max_depth=max_depth_factor,
            min_samples_split=self.min_samples_split,
            min_samples_leaf=self.min_samples_leaf,
            max_leaf_nodes=self.max_leaf_nodes,
            min_weight_fraction_leaf=self.min_weight_fraction_leaf,
            min_impurity_decrease=self.min_impurity_decrease,
            random_state=self.random_state)

        if y.ndim == 2 and y.shape[1] == 1:
            y = y.flatten()

        self.estimator.fit(X, y, sample_weight=sample_weight)
        return self

    def predict(self, X):
        if self.estimator is None:
            raise NotImplementedError
        return self.estimator.predict(X)

    @staticmethod
    def get_properties(dataset_properties=None):
        return {'shortname': 'DT',
                'name': 'Decision Tree Classifier',
                'handles_regression': True,
                'handles_classification': False,
                'handles_multiclass': False,
                'handles_multilabel': False,
                'handles_multioutput': True,
                'is_deterministic': False,
                'input': (DENSE, SPARSE, UNSIGNED_DATA),
                'output': (PREDICTIONS,)}

    @staticmethod
    def get_hyperparameter_search_space(dataset_properties=None):
        cs = ConfigurationSpace()

        criterion = CategoricalHyperparameter('criterion',
                                              ['mse', 'friedman_mse', 'mae'])
        max_features = Constant('max_features', 1.0)
        max_depth_factor = UniformFloatHyperparameter(
            'max_depth_factor', 0., 2., default_value=0.5)
        min_samples_split = UniformIntegerHyperparameter(
            "min_samples_split", 2, 20, default_value=2)
        min_samples_leaf = UniformIntegerHyperparameter(
            "min_samples_leaf", 1, 20, default_value=1)
        min_weight_fraction_leaf = Constant("min_weight_fraction_leaf", 0.0)
        max_leaf_nodes = UnParametrizedHyperparameter("max_leaf_nodes", "None")
        min_impurity_decrease = UnParametrizedHyperparameter(
            'min_impurity_decrease', 0.0)

        cs.add_hyperparameters([criterion, max_features, max_depth_factor,
                                min_samples_split, min_samples_leaf,
                                min_weight_fraction_leaf, max_leaf_nodes,
                                min_impurity_decrease])

        return cs

In [ ]:
# Second Step: Optimize Hyperparameter with automl.

reg = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=time_left_for_this_task,
    per_run_time_limit=per_run_time_limit,
    include={
        'regressor': ['decision_tree'],
        'feature_preprocessor': ['no_preprocessing']
    },
    ensemble_size=1
)
reg.fit(X_train, y_train)

[WARNING] [2022-06-29 09:07:00,157:Client-AutoMLSMBO(1)::d50cff6e-f78a-11ec-8204-0242ac1c0002] Configuration 197 not found
[WARNING] [2022-06-29 09:07:00,157:Client-AutoMLSMBO(1)::d50cff6e-f78a-11ec-8204-0242ac1c0002] Configuration 235 not found
[WARNING] [2022-06-29 09:07:00,157:Client-AutoMLSMBO(1)::d50cff6e-f78a-11ec-8204-0242ac1c0002] Configuration 270 not found
[WARNING] [2022-06-29 09:07:00,157:Client-AutoMLSMBO(1)::d50cff6e-f78a-11ec-8204-0242ac1c0002] Configuration 163 not found
[WARNING] [2022-06-29 09:07:00,157:Client-AutoMLSMBO(1)::d50cff6e-f78a-11ec-8204-0242ac1c0002] Configuration 67 not found
[WARNING] [2022-06-29 09:07:00,157:Client-AutoMLSMBO(1)::d50cff6e-f78a-11ec-8204-0242ac1c0002] Configuration 34 not found
[WARNING] [2022-06-29 09:07:00,157:Client-AutoMLSMBO(1)::d50cff6e-f78a-11ec-8204-0242ac1c0002] Configuration 86 not found
[WARNING] [2022-06-29 09:07:00,157:Client-AutoMLSMBO(1)::d50cff6e-f78a-11ec-8204-0242ac1c0002] Configuration 240 not found
[WARNING] [2022-06-

AutoSklearnRegressor(ensemble_size=1,
                     include={'feature_preprocessor': ['no_preprocessing'],
                              'regressor': ['decision_tree']},
                     per_run_time_limit=20, time_left_for_this_task=180)

In [ ]:
# Third Step: Save best params and best score.
model_dict = reg.show_models()
key = list(model_dict.keys())[0]
print(model_dict)

dt_best_params = [model_dict[key]['sklearn_regressor'].max_depth, model_dict[key]['sklearn_regressor'].min_samples_leaf, model_dict[key]['sklearn_regressor'].min_samples_split]
dt_best_score = model_dict[key]['cost']

print('Best Parameter for Max Depthl: {}. Best Parameter for Min Samples Leaf: {}. Best Parameter for Min Samples Split: {}. Best Score: {}'.format(dt_best_params[0], dt_best_params[1], dt_best_params[2], svr_best_score))
dtParams = model_dict[key]['sklearn_regressor'].get_params()

{71: {'model_id': 71, 'rank': 1, 'cost': 0.3278674427281554, 'ensemble_weight': 1.0, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f76e52c0210>, 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f76e523dc50>, 'regressor': <autosklearn.pipeline.components.regression.RegressorChoice object at 0x7f76e523de90>, 'sklearn_regressor': DecisionTreeRegressor(criterion='friedman_mse', max_depth=9, min_samples_leaf=4,
                      min_samples_split=4, random_state=1)}}
Best Parameter for Max Depthl: 9. Best Parameter for Min Samples Leaf: 4. Best Parameter for Min Samples Split: 4. Best Score: 0.02685871308620924


# Optimize Kernel Ridge Regression
Hyperparameters: alpha, Kernel, gamma, degree, coef0
available Kernels: Polynomial, RBF

In [ ]:
from ConfigSpace.configuration_space import ConfigurationSpace
from ConfigSpace.hyperparameters import UniformFloatHyperparameter, \
    UniformIntegerHyperparameter, CategoricalHyperparameter
from ConfigSpace.conditions import EqualsCondition

import sklearn.metrics
import autosklearn.regression
import autosklearn.pipeline.components.regression
from autosklearn.pipeline.components.base import AutoSklearnRegressionAlgorithm
from autosklearn.pipeline.constants import SPARSE, DENSE, \
    SIGNED_DATA, UNSIGNED_DATA, PREDICTIONS

class KernelRidgeRegression(AutoSklearnRegressionAlgorithm):

    def __init__(self, alpha, kernel, gamma, degree, coef0, random_state=None):
        self.alpha = alpha
        self.kernel = kernel
        self.gamma = gamma
        self.degree = degree
        self.coef0 = coef0
        self.random_state = random_state
        self.estimator = None

    def fit(self, X, y):
        self.alpha = float(self.alpha)
        self.gamma = float(self.gamma)
        self.degree = int(self.degree)
        self.coef0 = float(self.coef0)

        import sklearn.kernel_ridge
        self.estimator = sklearn.kernel_ridge.KernelRidge(
            alpha=self.alpha,
            kernel=self.kernel,
            gamma=self.gamma,
            degree=self.degree,
            coef0=self.coef0
        )
        self.estimator.fit(X, y)
        return self

    def predict(self, X):
        if self.estimator is None:
            raise NotImplementedError
        return self.estimator.predict(X)

    @staticmethod
    def get_properties(dataset_properties=None):
        return {
            'shortname': 'KRR',
            'name': 'Kernel Ridge Regression',
            'handles_regression': True,
            'handles_classification': False,
            'handles_multiclass': False,
            'handles_multilabel': False,
            'handles_multioutput': True,
            'is_deterministic': True,
            'input': (SPARSE, DENSE, UNSIGNED_DATA, SIGNED_DATA),
            'output': (PREDICTIONS,)
        }

    @staticmethod
    def get_hyperparameter_search_space(dataset_properties=None):
        cs = ConfigurationSpace()
        alpha = UniformFloatHyperparameter(
            name='alpha', lower=10 ** -5, upper=1, log=True, default_value=1.0
        )
        kernel = CategoricalHyperparameter(
            name='kernel',
            # We restrict ourselves to two possible kernels for this example
            choices=['polynomial', 'rbf'],
            default_value='polynomial'
        )
        gamma = UniformFloatHyperparameter(
            name='gamma', lower=0.00001, upper=1, default_value=0.1, log=True
        )
        degree = UniformIntegerHyperparameter(
            name='degree', lower=2, upper=5, default_value=3
        )
        coef0 = UniformFloatHyperparameter(
            name='coef0', lower=1e-2, upper=1e2, log=True, default_value=1,
        )
        cs.add_hyperparameters([alpha, kernel, gamma, degree, coef0])
        degree_condition = EqualsCondition(degree, kernel, 'polynomial')
        coef0_condition = EqualsCondition(coef0, kernel, 'polynomial')
        cs.add_conditions([degree_condition, coef0_condition])
        return cs


# Add KRR component to auto-sklearn.
autosklearn.pipeline.components.regression.add_regressor(KernelRidgeRegression)
cs = KernelRidgeRegression.get_hyperparameter_search_space()
print(cs)


Configuration space object:
  Hyperparameters:
    alpha, Type: UniformFloat, Range: [1e-05, 1.0], Default: 1.0, on log-scale
    coef0, Type: UniformFloat, Range: [0.01, 100.0], Default: 1.0, on log-scale
    degree, Type: UniformInteger, Range: [2, 5], Default: 3
    gamma, Type: UniformFloat, Range: [1e-05, 1.0], Default: 0.1, on log-scale
    kernel, Type: Categorical, Choices: {polynomial, rbf}, Default: polynomial
  Conditions:
    coef0 | kernel == 'polynomial'
    degree | kernel == 'polynomial'



In [ ]:
# Second Step: Optimize Hyperparameter with automl.

reg = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=time_left_for_this_task,
    per_run_time_limit=per_run_time_limit,
    include={
        'regressor': ['KernelRidgeRegression'],
        'feature_preprocessor': ['no_preprocessing']
    },
    ensemble_size=1
)
reg.fit(X_train, y_train)

[WARNING] [2022-06-29 09:09:54,154:Client-AutoMLSMBO(1)::3ca48a76-f78b-11ec-8204-0242ac1c0002] Configuration 197 not found
[WARNING] [2022-06-29 09:09:54,154:Client-AutoMLSMBO(1)::3ca48a76-f78b-11ec-8204-0242ac1c0002] Configuration 235 not found
[WARNING] [2022-06-29 09:09:54,154:Client-AutoMLSMBO(1)::3ca48a76-f78b-11ec-8204-0242ac1c0002] Configuration 270 not found
[WARNING] [2022-06-29 09:09:54,154:Client-AutoMLSMBO(1)::3ca48a76-f78b-11ec-8204-0242ac1c0002] Configuration 163 not found
[WARNING] [2022-06-29 09:09:54,154:Client-AutoMLSMBO(1)::3ca48a76-f78b-11ec-8204-0242ac1c0002] Configuration 67 not found
[WARNING] [2022-06-29 09:09:54,154:Client-AutoMLSMBO(1)::3ca48a76-f78b-11ec-8204-0242ac1c0002] Configuration 34 not found
[WARNING] [2022-06-29 09:09:54,154:Client-AutoMLSMBO(1)::3ca48a76-f78b-11ec-8204-0242ac1c0002] Configuration 86 not found
[WARNING] [2022-06-29 09:09:54,154:Client-AutoMLSMBO(1)::3ca48a76-f78b-11ec-8204-0242ac1c0002] Configuration 240 not found
[WARNING] [2022-06-

AutoSklearnRegressor(ensemble_size=1,
                     include={'feature_preprocessor': ['no_preprocessing'],
                              'regressor': ['KernelRidgeRegression']},
                     per_run_time_limit=20, time_left_for_this_task=180)

In [ ]:
# Third Step: Save best params and best score.
model_dict = reg.show_models()
key = list(model_dict.keys())[0]
print(model_dict)
krr_best_params = [model_dict[key]['sklearn_regressor'].alpha, model_dict[key]['sklearn_regressor'].coef0, model_dict[key]['sklearn_regressor'].degree, model_dict[key]['sklearn_regressor'].gamma, model_dict[key]['sklearn_regressor'].kernel]
krr_best_score = model_dict[key]['cost']

print('Best Parameter for Max alpha: {}. Best Parameter for coef0: {}. Best Parameter for gamma: {}. Best Parameter for Kernel: {}. Best Score: {}'.format(krr_best_params[0], krr_best_params[1], krr_best_params[2], krr_best_params[3], krr_best_score))
krrParams = model_dict[key]['sklearn_regressor'].get_params()

{52: {'model_id': 52, 'rank': 1, 'cost': 0.0066054397868208214, 'ensemble_weight': 1.0, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f76e606dd10>, 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f76e5f136d0>, 'regressor': <autosklearn.pipeline.components.regression.RegressorChoice object at 0x7f76e5f13910>, 'sklearn_regressor': KernelRidge(alpha=0.000225416803944748, coef0=0.8885883850159546, degree=2,
            gamma=0.00012450644463786435, kernel='polynomial')}}
Best Parameter for Max alpha: 0.000225416803944748. Best Parameter for coef0: 0.8885883850159546. Best Parameter for gamma: 2. Best Parameter for Kernel: 0.00012450644463786435. Best Score: 0.0066054397868208214


In [ ]:
krr_best_params = [model_dict[key]['sklearn_regressor'].alpha, model_dict[key]['sklearn_regressor'].coef0, model_dict[key]['sklearn_regressor'].degree, model_dict[key]['sklearn_regressor'].gamma, model_dict[key]['sklearn_regressor'].kernel]
krr_best_score = model_dict[key]['cost']

print('Best Parameter for Max alpha: {}. Best Parameter for coef0: {}. Best Parameter for gamma: {}. Best Parameter for Kernel: {}. Best Score: {}'.format(krr_best_params[0], krr_best_params[1], krr_best_params[2], krr_best_params[3], krr_best_score))

Best Parameter for Max alpha: 0.000225416803944748. Best Parameter for coef0: 0.8885883850159546. Best Parameter for gamma: 2. Best Parameter for Kernel: 0.00012450644463786435. Best Score: 0.0066054397868208214


# Optimize kNN
Hyperparameters: Number of Neighbours, weights

In [ ]:
# Second Step: Optimize Hyperparameter with automl.

reg = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=time_left_for_this_task,
    per_run_time_limit=per_run_time_limit,
    include={
        'regressor': ['k_nearest_neighbors'],
        'feature_preprocessor': ['no_preprocessing']
    },
    ensemble_size=1
)
reg.fit(X_train, y_train)

[WARNING] [2022-06-29 09:12:48,373:Client-AutoMLSMBO(1)::a4868274-f78b-11ec-8204-0242ac1c0002] Configuration 197 not found
[WARNING] [2022-06-29 09:12:48,374:Client-AutoMLSMBO(1)::a4868274-f78b-11ec-8204-0242ac1c0002] Configuration 235 not found
[WARNING] [2022-06-29 09:12:48,374:Client-AutoMLSMBO(1)::a4868274-f78b-11ec-8204-0242ac1c0002] Configuration 270 not found
[WARNING] [2022-06-29 09:12:48,374:Client-AutoMLSMBO(1)::a4868274-f78b-11ec-8204-0242ac1c0002] Configuration 163 not found
[WARNING] [2022-06-29 09:12:48,374:Client-AutoMLSMBO(1)::a4868274-f78b-11ec-8204-0242ac1c0002] Configuration 67 not found
[WARNING] [2022-06-29 09:12:48,374:Client-AutoMLSMBO(1)::a4868274-f78b-11ec-8204-0242ac1c0002] Configuration 34 not found
[WARNING] [2022-06-29 09:12:48,374:Client-AutoMLSMBO(1)::a4868274-f78b-11ec-8204-0242ac1c0002] Configuration 86 not found
[WARNING] [2022-06-29 09:12:48,374:Client-AutoMLSMBO(1)::a4868274-f78b-11ec-8204-0242ac1c0002] Configuration 240 not found
[WARNING] [2022-06-

AutoSklearnRegressor(ensemble_size=1,
                     include={'feature_preprocessor': ['no_preprocessing'],
                              'regressor': ['k_nearest_neighbors']},
                     per_run_time_limit=20, time_left_for_this_task=180)

In [ ]:
# Third Step: Save best params and best score.
model_dict = reg.show_models()
key = list(model_dict.keys())[0]
print(model_dict)
knn_best_params = [model_dict[key]['sklearn_regressor'].n_neighbors, model_dict[key]['sklearn_regressor'].weights]
knn_best_score = model_dict[key]['cost']

print('Best Parameter for Number of Neighbours: {}. Best Parameter for Weights: {}. Best Score: {}'.format(knn_best_params[0], knn_best_params[1], knn_best_score))
knnParams = model_dict[key]['sklearn_regressor'].get_params()

{69: {'model_id': 69, 'rank': 1, 'cost': 0.12011981819173356, 'ensemble_weight': 1.0, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f76e5c53810>, 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f76e5d39810>, 'regressor': <autosklearn.pipeline.components.regression.RegressorChoice object at 0x7f76e5fa5690>, 'sklearn_regressor': KNeighborsRegressor(n_neighbors=2, p=1, weights='distance')}}
Best Parameter for Number of Neighbours: 2. Best Parameter for Weights: distance. Best Score: 0.12011981819173356


# Optimize Random Forest
Hyperparameters: criterion, max_features,
                                max_depth, min_samples_split, min_samples_leaf,
                                min_weight_fraction_leaf, max_leaf_nodes,
                                min_impurity_decrease, bootstrap


In [ ]:
# Second Step: Optimize Hyperparameter with automl.

reg = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=time_left_for_this_task,
    per_run_time_limit=per_run_time_limit,
    include={
        'regressor': ['random_forest'],
        'feature_preprocessor': ['no_preprocessing']
    },
    ensemble_size=1
)
reg.fit(X_train, y_train)

[WARNING] [2022-06-29 09:15:43,543:Client-AutoMLSMBO(1)::0caf64c2-f78c-11ec-8204-0242ac1c0002] Configuration 197 not found
[WARNING] [2022-06-29 09:15:43,543:Client-AutoMLSMBO(1)::0caf64c2-f78c-11ec-8204-0242ac1c0002] Configuration 235 not found
[WARNING] [2022-06-29 09:15:43,543:Client-AutoMLSMBO(1)::0caf64c2-f78c-11ec-8204-0242ac1c0002] Configuration 270 not found
[WARNING] [2022-06-29 09:15:43,543:Client-AutoMLSMBO(1)::0caf64c2-f78c-11ec-8204-0242ac1c0002] Configuration 163 not found
[WARNING] [2022-06-29 09:15:43,543:Client-AutoMLSMBO(1)::0caf64c2-f78c-11ec-8204-0242ac1c0002] Configuration 67 not found
[WARNING] [2022-06-29 09:15:43,543:Client-AutoMLSMBO(1)::0caf64c2-f78c-11ec-8204-0242ac1c0002] Configuration 34 not found
[WARNING] [2022-06-29 09:15:43,543:Client-AutoMLSMBO(1)::0caf64c2-f78c-11ec-8204-0242ac1c0002] Configuration 86 not found
[WARNING] [2022-06-29 09:15:43,543:Client-AutoMLSMBO(1)::0caf64c2-f78c-11ec-8204-0242ac1c0002] Configuration 240 not found
[WARNING] [2022-06-

AutoSklearnRegressor(ensemble_size=1,
                     include={'feature_preprocessor': ['no_preprocessing'],
                              'regressor': ['random_forest']},
                     per_run_time_limit=20, time_left_for_this_task=180)

In [ ]:
# Third Step: Save best params and best score.
model_dict = reg.show_models()
key = list(model_dict.keys())[0]
print(model_dict)

rf_best_params = [model_dict[key]['sklearn_regressor'].criterion, model_dict[key]['sklearn_regressor'].max_features, model_dict[key]['sklearn_regressor'].n_estimators]
rf_best_score = model_dict[key]['cost']

print('Best Parameter for Criterion: {}. Best Parameter for max features: {}. Best Parameter for n_estimators: {}. Best Score: {}'.format(rf_best_params[0], rf_best_params[1], rf_best_params[2], rf_best_score))
rfParams = model_dict[key]['sklearn_regressor'].get_params()

{62: {'model_id': 62, 'rank': 1, 'cost': 0.29894862903211494, 'ensemble_weight': 1.0, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f76e5c3f590>, 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f76e5c21490>, 'regressor': <autosklearn.pipeline.components.regression.RegressorChoice object at 0x7f76e51b2350>, 'sklearn_regressor': RandomForestRegressor(bootstrap=False, max_features=0.6796999131064536,
                      min_samples_leaf=3, min_samples_split=3, n_estimators=512,
                      n_jobs=1, random_state=1, warm_start=True)}}
Best Parameter for Criterion: mse. Best Parameter for max features: 0.6796999131064536. Best Parameter for n_estimators: 512. Best Score: 0.29894862903211494


# Optimize Extra Trees Regressor

Hyperparameters: criterion, max_features,
                                max_depth, max_leaf_nodes, min_samples_split,
                                min_samples_leaf, min_impurity_decrease, min_weight_fraction_leaf,
                                bootstrap

In [ ]:
from ConfigSpace.configuration_space import ConfigurationSpace
from ConfigSpace.hyperparameters import UniformFloatHyperparameter, \
    UniformIntegerHyperparameter, CategoricalHyperparameter, UnParametrizedHyperparameter

from autosklearn.pipeline.components.base import (
    AutoSklearnRegressionAlgorithm,
    IterativeComponent,
)
from autosklearn.pipeline.constants import DENSE, UNSIGNED_DATA, PREDICTIONS, SPARSE
from autosklearn.util.common import check_for_bool, check_none


class ExtraTreesRegressorExtendedAutoML(
    IterativeComponent,
    AutoSklearnRegressionAlgorithm,
):
    def __init__(self, criterion, min_samples_leaf,
                 min_samples_split, max_features, bootstrap, max_leaf_nodes,
                 max_depth, min_weight_fraction_leaf, min_impurity_decrease,
                 oob_score=False, n_jobs=1, random_state=None, verbose=0):

        self.n_estimators = self.get_max_iter()
        self.criterion = criterion
        self.max_leaf_nodes = max_leaf_nodes
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.bootstrap = bootstrap
        self.max_depth = max_depth
        self.min_weight_fraction_leaf = min_weight_fraction_leaf
        self.min_impurity_decrease = min_impurity_decrease
        self.oob_score = oob_score
        self.n_jobs = n_jobs
        self.random_state = random_state
        self.verbose = verbose
        self.estimator = None

    @staticmethod
    def get_max_iter():
        return 512

    def get_current_iter(self):
        return self.estimator.n_estimators

    def iterative_fit(self, X, y, n_iter=1, refit=False):
        from sklearn.ensemble import ExtraTreesRegressor as ETR

        if refit:
            self.estimator = None

        if self.estimator is None:
            self.n_estimators = int(self.n_estimators)
            if self.criterion not in ("mse", "friedman_mse", "mae"):
                raise ValueError(
                    "'criterion' is not in ('mse', 'friedman_mse', "
                    "'mae): %s" % self.criterion)

            if check_none(self.max_depth):
                self.max_depth = None
            else:
                self.max_depth = int(self.max_depth)

            if check_none(self.max_leaf_nodes):
                self.max_leaf_nodes = None
            else:
                self.max_leaf_nodes = int(self.max_leaf_nodes)

            self.min_samples_leaf = int(self.min_samples_leaf)
            self.min_samples_split = int(self.min_samples_split)
            self.max_features = float(self.max_features)
            self.min_impurity_decrease = float(self.min_impurity_decrease)
            self.min_weight_fraction_leaf = float(self.min_weight_fraction_leaf)
            self.oob_score = check_for_bool(self.oob_score)
            self.bootstrap = check_for_bool(self.bootstrap)
            self.n_jobs = int(self.n_jobs)
            self.verbose = int(self.verbose)

            self.estimator = ETR(n_estimators=n_iter,
                                 criterion=self.criterion,
                                 max_depth=self.max_depth,
                                 min_samples_split=self.min_samples_split,
                                 min_samples_leaf=self.min_samples_leaf,
                                 bootstrap=self.bootstrap,
                                 max_features=self.max_features,
                                 max_leaf_nodes=self.max_leaf_nodes,
                                 min_weight_fraction_leaf=self.min_weight_fraction_leaf,
                                 min_impurity_decrease=self.min_impurity_decrease,
                                 oob_score=self.oob_score,
                                 n_jobs=self.n_jobs,
                                 verbose=self.verbose,
                                 random_state=self.random_state,
                                 warm_start=True)
        else:
            self.estimator.n_estimators += n_iter
            self.estimator.n_estimators = min(self.estimator.n_estimators,
                                              self.n_estimators)

        if y.ndim == 2 and y.shape[1] == 1:
            y = y.flatten()

        self.estimator.fit(X, y)

        return self

    def configuration_fully_fitted(self):
        if self.estimator is None:
            return False
        return not len(self.estimator.estimators_) < self.n_estimators

    def predict(self, X):
        if self.estimator is None:
            raise NotImplementedError
        return self.estimator.predict(X)

    @staticmethod
    def get_properties(dataset_properties=None):
        return {'shortname': 'ET',
                'name': 'Extra Trees Regressor',
                'handles_regression': True,
                'handles_classification': False,
                'handles_multiclass': False,
                'handles_multilabel': False,
                'handles_multioutput': True,
                'is_deterministic': True,
                'input': (DENSE, SPARSE, UNSIGNED_DATA),
                'output': (PREDICTIONS,)}

    @staticmethod
    def get_hyperparameter_search_space(dataset_properties=None):
        cs = ConfigurationSpace()

        criterion = CategoricalHyperparameter("criterion",
                                              ['mse', 'friedman_mse', 'mae'])
        max_features = UniformFloatHyperparameter(
            "max_features", 0.1, 1.0, default_value=1)

        max_depth = UnParametrizedHyperparameter(name="max_depth", value="None")
        min_weight_fraction_leaf = UnParametrizedHyperparameter('min_weight_fraction_leaf', 0.)
        max_leaf_nodes = UnParametrizedHyperparameter("max_leaf_nodes", "None")

        min_samples_split = UniformIntegerHyperparameter(
            "min_samples_split", 2, 20, default_value=2)
        min_samples_leaf = UniformIntegerHyperparameter(
            "min_samples_leaf", 1, 20, default_value=1)
        min_impurity_decrease = UnParametrizedHyperparameter(
            'min_impurity_decrease', 0.0
        )

        bootstrap = CategoricalHyperparameter(
            "bootstrap", ["True", "False"], default_value="False")

        cs.add_hyperparameters([criterion, max_features,
                                max_depth, max_leaf_nodes, min_samples_split,
                                min_samples_leaf, min_impurity_decrease, min_weight_fraction_leaf,
                                bootstrap])

        return cs

autosklearn.pipeline.components.regression.add_regressor(ExtraTreesRegressorExtendedAutoML)
cs = ExtraTreesRegressorExtendedAutoML.get_hyperparameter_search_space()
print(cs)


Configuration space object:
  Hyperparameters:
    bootstrap, Type: Categorical, Choices: {True, False}, Default: False
    criterion, Type: Categorical, Choices: {mse, friedman_mse, mae}, Default: mse
    max_depth, Type: Constant, Value: None
    max_features, Type: UniformFloat, Range: [0.1, 1.0], Default: 1.0
    max_leaf_nodes, Type: Constant, Value: None
    min_impurity_decrease, Type: Constant, Value: 0.0
    min_samples_leaf, Type: UniformInteger, Range: [1, 20], Default: 1
    min_samples_split, Type: UniformInteger, Range: [2, 20], Default: 2
    min_weight_fraction_leaf, Type: Constant, Value: 0.0



In [ ]:
# Second Step: Optimize Hyperparameter with automl.

reg = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=time_left_for_this_task,
    per_run_time_limit=per_run_time_limit,
    include={
        'regressor': ['ExtraTreesRegressorExtendedAutoML'],
        'feature_preprocessor': ['no_preprocessing']
    },
    ensemble_size=1
)
reg.fit(X_train, y_train)

[WARNING] [2022-06-29 09:45:10,524:Client-AutoMLSMBO(1)::29b31834-f790-11ec-8204-0242ac1c0002] Configuration 197 not found
[WARNING] [2022-06-29 09:45:10,524:Client-AutoMLSMBO(1)::29b31834-f790-11ec-8204-0242ac1c0002] Configuration 235 not found
[WARNING] [2022-06-29 09:45:10,524:Client-AutoMLSMBO(1)::29b31834-f790-11ec-8204-0242ac1c0002] Configuration 270 not found
[WARNING] [2022-06-29 09:45:10,524:Client-AutoMLSMBO(1)::29b31834-f790-11ec-8204-0242ac1c0002] Configuration 163 not found
[WARNING] [2022-06-29 09:45:10,524:Client-AutoMLSMBO(1)::29b31834-f790-11ec-8204-0242ac1c0002] Configuration 67 not found
[WARNING] [2022-06-29 09:45:10,524:Client-AutoMLSMBO(1)::29b31834-f790-11ec-8204-0242ac1c0002] Configuration 34 not found
[WARNING] [2022-06-29 09:45:10,524:Client-AutoMLSMBO(1)::29b31834-f790-11ec-8204-0242ac1c0002] Configuration 86 not found
[WARNING] [2022-06-29 09:45:10,525:Client-AutoMLSMBO(1)::29b31834-f790-11ec-8204-0242ac1c0002] Configuration 240 not found
[WARNING] [2022-06-

AutoSklearnRegressor(ensemble_size=1,
                     include={'feature_preprocessor': ['no_preprocessing'],
                              'regressor': ['ExtraTreesRegressorExtendedAutoML']},
                     per_run_time_limit=20, time_left_for_this_task=180)

In [ ]:
# Third Step: Save best params and best score.
model_dict = reg.show_models()
key = list(model_dict.keys())[0]
print(model_dict)

et_best_params = [model_dict[key]['sklearn_regressor'].criterion, model_dict[key]['sklearn_regressor'].max_features, model_dict[key]['sklearn_regressor'].min_samples_split, model_dict[key]['sklearn_regressor'].n_estimators]
et_best_score = model_dict[key]['cost']

print('Best Parameter for Criterion: {}. Best Parameter for max features: {}. Best Parameter for min_samples_split: {} Best Parameter for n_estimators: {}. Best Score: {}'.format(et_best_params[0], et_best_params[1], et_best_params[2], et_best_params[3], et_best_score))
etParams = model_dict[key]['sklearn_regressor'].get_params()
#del etParams['ccp_alpha']

{56: {'model_id': 56, 'rank': 1, 'cost': 0.11600665626463869, 'ensemble_weight': 1.0, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f76e4ba2f10>, 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f76e60d1b90>, 'regressor': <autosklearn.pipeline.components.regression.RegressorChoice object at 0x7f76e60d1790>, 'sklearn_regressor': ExtraTreesRegressor(criterion='friedman_mse', max_features=0.8911502982083459,
                    n_estimators=512, n_jobs=1, random_state=1,
                    warm_start=True)}}
Best Parameter for Criterion: friedman_mse. Best Parameter for max features: 0.8911502982083459. Best Parameter for min_samples_split: 2 Best Parameter for n_estimators: 512. Best Score: 0.11600665626463869


In [ ]:
ExtraTreesRegressor(**etParams)

TypeError: ignored

# Optimize adaBoost Regressor
Parameters: n_estimators, learning_rate, loss, max_depth

In [ ]:
# Second Step: Optimize Hyperparameter with automl.

reg = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=time_left_for_this_task,
    per_run_time_limit=per_run_time_limit,
    include={
        'regressor': ['adaboost'],
        'feature_preprocessor': ['no_preprocessing']
    },
    ensemble_size=1
)
reg.fit(X_train, y_train)

[WARNING] [2022-06-29 09:21:32,357:Client-AutoMLSMBO(1)::dcf992ee-f78c-11ec-8204-0242ac1c0002] Configuration 197 not found
[WARNING] [2022-06-29 09:21:32,357:Client-AutoMLSMBO(1)::dcf992ee-f78c-11ec-8204-0242ac1c0002] Configuration 235 not found
[WARNING] [2022-06-29 09:21:32,357:Client-AutoMLSMBO(1)::dcf992ee-f78c-11ec-8204-0242ac1c0002] Configuration 270 not found
[WARNING] [2022-06-29 09:21:32,357:Client-AutoMLSMBO(1)::dcf992ee-f78c-11ec-8204-0242ac1c0002] Configuration 163 not found
[WARNING] [2022-06-29 09:21:32,357:Client-AutoMLSMBO(1)::dcf992ee-f78c-11ec-8204-0242ac1c0002] Configuration 67 not found
[WARNING] [2022-06-29 09:21:32,357:Client-AutoMLSMBO(1)::dcf992ee-f78c-11ec-8204-0242ac1c0002] Configuration 34 not found
[WARNING] [2022-06-29 09:21:32,357:Client-AutoMLSMBO(1)::dcf992ee-f78c-11ec-8204-0242ac1c0002] Configuration 86 not found
[WARNING] [2022-06-29 09:21:32,357:Client-AutoMLSMBO(1)::dcf992ee-f78c-11ec-8204-0242ac1c0002] Configuration 240 not found
[WARNING] [2022-06-

AutoSklearnRegressor(ensemble_size=1,
                     include={'feature_preprocessor': ['no_preprocessing'],
                              'regressor': ['adaboost']},
                     per_run_time_limit=20, time_left_for_this_task=180)

In [ ]:
# Third Step: Save best params and best score.
model_dict = reg.show_models()
key = list(model_dict.keys())[0]
print(model_dict)

ab_best_params = [model_dict[key]['sklearn_regressor'].base_estimator, model_dict[key]['sklearn_regressor'].learning_rate, model_dict[key]['sklearn_regressor'].n_estimators]
ab_best_score = model_dict[key]['cost']


print('Best Parameter for base estimator: {}. Best Parameter for learning rate: {}. Best Parameter for n_estimators: {}. Best Score: {}'.format(ab_best_params[0], ab_best_params[1], ab_best_params[2], ab_best_score))
abParams = model_dict[key]['sklearn_regressor'].get_params()

{77: {'model_id': 77, 'rank': 1, 'cost': 0.20793052936831602, 'ensemble_weight': 1.0, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f76e5bd6810>, 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f76e60d4490>, 'regressor': <autosklearn.pipeline.components.regression.RegressorChoice object at 0x7f76e5b91390>, 'sklearn_regressor': AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=7),
                  learning_rate=0.011330528524456526, n_estimators=61,
                  random_state=1)}}
Best Parameter for base estimator: DecisionTreeRegressor(max_depth=7). Best Parameter for learning rate: 0.011330528524456526. Best Parameter for n_estimators: 61. Best Score: 0.20793052936831602


# Optimize GradientBoosting
Hyperparameters: n_estimators, learning_rate,
                                   max_depth, max_features,
                                   min_samples_leaf, min_samples_split, 
                                   loss

In [ ]:
# First Step: Extend autosklearn

from ConfigSpace.configuration_space import ConfigurationSpace
from ConfigSpace.hyperparameters import UniformFloatHyperparameter, \
    UniformIntegerHyperparameter, CategoricalHyperparameter
from ConfigSpace.conditions import EqualsCondition

import sklearn.metrics
import autosklearn.regression
import autosklearn.pipeline.components.regression
from autosklearn.pipeline.components.base import AutoSklearnRegressionAlgorithm
from autosklearn.pipeline.constants import SPARSE, DENSE, \
    SIGNED_DATA, UNSIGNED_DATA, PREDICTIONS

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

class GradientBoostingExtendedAutoML(AutoSklearnRegressionAlgorithm):

    def __init__(self, n_estimators, learning_rate, max_depth, max_features, min_samples_leaf, min_samples_split, loss, random_state=1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.max_features = max_features
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_split = min_samples_split
        self.loss = loss
        self.random_state = random_state
        self.estimator = None

    def fit(self, X, y):
        self.n_estimators = int(self.n_estimators)
        self.learning_rate = float(self.learning_rate)
        self.max_depth = int(self.max_depth)

        from sklearn.ensemble import GradientBoostingRegressor
        self.estimator = GradientBoostingRegressor(
            n_estimators=self.n_estimators, 
            learning_rate=self.learning_rate,
            max_depth=self.max_depth, 
            max_features=self.max_features,
            min_samples_leaf=self.min_samples_leaf,
            min_samples_split=self.min_samples_split,
            loss=self.loss,
            random_state=self.random_state
        )
        self.estimator.fit(X, y)
        return self

    def predict(self, X):
        if self.estimator is None:
            raise NotImplementedError
        return self.estimator.predict(X)

    @staticmethod
    def get_properties(dataset_properties=None):
        return {
            'shortname': 'GB',
            'name': 'Gradient Boosting Regression',
            'handles_regression': True,
            'handles_classification': False,
            'handles_multiclass': False,
            'handles_multilabel': False,
            'handles_multioutput': True,
            'is_deterministic': True,
            'input': (SPARSE, DENSE, UNSIGNED_DATA, SIGNED_DATA),
            'output': (PREDICTIONS,)
        }

    @staticmethod
    def get_hyperparameter_search_space(dataset_properties=None):
        cs = ConfigurationSpace()
        # The number of boosting stages to perform. Gradient boosting is fairly robust to over-fitting so a large number usually results in better performance. 
        # Upper Bound is an ad hoc choice.
        n_estimators = UniformIntegerHyperparameter(
            name='n_estimators', lower=1, upper=10000, log=True, default_value=100
        )
        # Learning rate shrinks the contribution of each tree by learning_rate. There is a trade-off between learning_rate and n_estimators. Values must be in the range (0.0, inf).
        learning_rate = UniformFloatHyperparameter(
            name='learning_rate', lower= 10**-5, upper=10000, log=True, default_value=0.1
        )
        # Maximum depth of the individual regression estimators. The maximum depth limits the number of nodes in the tree. Tune this parameter for best performance; the best value depends on the interaction of the input variables. Values must be in the range [1, inf).
        max_depth = UniformIntegerHyperparameter(
            name='max_depth', lower=1, upper=100, log=True, default_value=3
        )
        max_features = UniformIntegerHyperparameter(
            name='max_features', lower=1, upper=100, log=True, default_value=3
        )
        min_samples_leaf = UniformFloatHyperparameter(
            name='min_samples_leaf', lower= 10**-5, upper=1, log=True, default_value=1
        )
        min_samples_split = UniformFloatHyperparameter(
            name='min_samples_split', lower= 10**-5, upper=1, log=True, default_value=1
        )
        loss = CategoricalHyperparameter(
            name='loss',
            choices=['squared_error', 'absolute_error', 'huber', 'quantile'],
            default_value='squared_error'
        )        
        cs.add_hyperparameters([n_estimators, learning_rate, max_depth, max_features, min_samples_leaf, min_samples_split, loss])
        return cs


# Add GBR component to auto-sklearn.
autosklearn.pipeline.components.regression.add_regressor(GradientBoostingExtendedAutoML)
cs = GradientBoostingExtendedAutoML.get_hyperparameter_search_space()
print(cs)


Configuration space object:
  Hyperparameters:
    learning_rate, Type: UniformFloat, Range: [1e-05, 10000.0], Default: 0.1, on log-scale
    loss, Type: Categorical, Choices: {squared_error, absolute_error, huber, quantile}, Default: squared_error
    max_depth, Type: UniformInteger, Range: [1, 100], Default: 3, on log-scale
    max_features, Type: UniformInteger, Range: [1, 100], Default: 3, on log-scale
    min_samples_leaf, Type: UniformFloat, Range: [1e-05, 1.0], Default: 1.0, on log-scale
    min_samples_split, Type: UniformFloat, Range: [1e-05, 1.0], Default: 1.0, on log-scale
    n_estimators, Type: UniformInteger, Range: [1, 10000], Default: 100, on log-scale



In [ ]:
# Second Step: Optimize Hyperparameter with automl.

reg = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=time_left_for_this_task,
    per_run_time_limit=per_run_time_limit,
    include={
        'regressor': ['GradientBoostingExtendedAutoML'],
        'feature_preprocessor': ['no_preprocessing']
    },
    ensemble_size=1
)
reg.fit(X_train, y_train)

[WARNING] [2022-06-29 10:02:04,678:Client-AutoMLSMBO(1)::864329c0-f792-11ec-8204-0242ac1c0002] Configuration 197 not found
[WARNING] [2022-06-29 10:02:04,678:Client-AutoMLSMBO(1)::864329c0-f792-11ec-8204-0242ac1c0002] Configuration 235 not found
[WARNING] [2022-06-29 10:02:04,678:Client-AutoMLSMBO(1)::864329c0-f792-11ec-8204-0242ac1c0002] Configuration 270 not found
[WARNING] [2022-06-29 10:02:04,678:Client-AutoMLSMBO(1)::864329c0-f792-11ec-8204-0242ac1c0002] Configuration 163 not found
[WARNING] [2022-06-29 10:02:04,678:Client-AutoMLSMBO(1)::864329c0-f792-11ec-8204-0242ac1c0002] Configuration 67 not found
[WARNING] [2022-06-29 10:02:04,678:Client-AutoMLSMBO(1)::864329c0-f792-11ec-8204-0242ac1c0002] Configuration 34 not found
[WARNING] [2022-06-29 10:02:04,678:Client-AutoMLSMBO(1)::864329c0-f792-11ec-8204-0242ac1c0002] Configuration 86 not found
[WARNING] [2022-06-29 10:02:04,678:Client-AutoMLSMBO(1)::864329c0-f792-11ec-8204-0242ac1c0002] Configuration 240 not found
[WARNING] [2022-06-

AutoSklearnRegressor(ensemble_size=1,
                     include={'feature_preprocessor': ['no_preprocessing'],
                              'regressor': ['GradientBoostingExtendedAutoML']},
                     per_run_time_limit=20, time_left_for_this_task=180)

In [ ]:
# Third Step: Save best params and best score.
model_dict = reg.show_models()
key = list(model_dict.keys())[0]
print(model_dict)


#print('Best Parameter for l2 regularizer: {}. Best Parameter for learning rate: {}. Best Parameter for max_iter: {}. Best Parameter for max leaf nodes: {}. Best Parameter for min samples leaf: {}. Best Parameter for n_iter_no_change: {}. Best Score: {}'.format(gb_best_params[0], gb_best_params[1], gb_best_params[2], gb_best_params[3], gb_best_params[4], gb_best_params[5], gb_best_score))

gbParams = model_dict[key]['sklearn_regressor'].get_params()

{9: {'model_id': 9, 'rank': 1, 'cost': 0.2528367263123297, 'ensemble_weight': 1.0, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f76e5b90290>, 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f76e5f90950>, 'regressor': <autosklearn.pipeline.components.regression.RegressorChoice object at 0x7f76e5b5e510>, 'sklearn_regressor': GradientBoostingRegressor(learning_rate=0.30542946987586317, loss='huber',
                          max_depth=62, max_features=6,
                          min_samples_leaf=0.02637164191839266,
                          min_samples_split=7.115963050600792e-05,
                          n_estimators=2467, random_state=1)}}


# Optimize XGBoost
Hyperparameters: colsample_bytree, gamma, 
                             learning_rate, max_depth, 
                             min_child_weight, n_estimators,
                             reg_alpha, reg_lambda,
                             subsample

In [ ]:
# First Step: Extend autosklearn

from ConfigSpace.configuration_space import ConfigurationSpace
from ConfigSpace.hyperparameters import UniformFloatHyperparameter, \
    UniformIntegerHyperparameter, CategoricalHyperparameter
from ConfigSpace.conditions import EqualsCondition

import sklearn.metrics
import autosklearn.regression
import autosklearn.pipeline.components.regression
from autosklearn.pipeline.components.base import AutoSklearnRegressionAlgorithm
from autosklearn.pipeline.constants import SPARSE, DENSE, \
    SIGNED_DATA, UNSIGNED_DATA, PREDICTIONS

class XGBoostRegression(AutoSklearnRegressionAlgorithm):

    def __init__(self, colsample_bytree, gamma, learning_rate, max_depth, min_child_weight, n_estimators, reg_alpha, reg_lambda, subsample, nthread=-1, random_state=1):
        self.colsample_bytree = colsample_bytree
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_child_weight = min_child_weight
        self.n_estimators = n_estimators
        self.reg_alpha = reg_alpha
        self.reg_lambda = reg_lambda
        self.subsample = subsample
        self.nthread = nthread
        self.random_state = random_state
        self.estimator = None

    def fit(self, X, y):
        from xgboost import XGBRegressor
        self.estimator = XGBRegressor(
            colsample_bytree = self.colsample_bytree,
            gamma = self.gamma,
            learning_rate = self.learning_rate,
            max_depth = self.max_depth,
            min_child_weight = self.min_child_weight,
            n_estimators = self.n_estimators,
            reg_alpha = self.reg_alpha,
            reg_lambda = self.reg_lambda,
            subsample = self.subsample,
            nthread = self.nthread,
            random_state = self.random_state
        )
        self.estimator.fit(X, y)
        return self

    def predict(self, X):
        if self.estimator is None:
            raise NotImplementedError
        return self.estimator.predict(X)

    @staticmethod
    def get_properties(dataset_properties=None):
        return {
            'shortname': 'XGB',
            'name': 'XGBoostRegression',
            'handles_regression': True,
            'handles_classification': False,
            'handles_multiclass': False,
            'handles_multilabel': False,
            'handles_multioutput': True,
            'is_deterministic': True,
            'input': (SPARSE, DENSE, UNSIGNED_DATA, SIGNED_DATA),
            'output': (PREDICTIONS,)
        }

    @staticmethod
    def get_hyperparameter_search_space(dataset_properties=None):
        cs = ConfigurationSpace()
        colsample_bytree = UniformFloatHyperparameter(
            name='colsample_bytree', lower=10 ** -5, upper=1, log=True, default_value=1
        )
        gamma = UniformFloatHyperparameter(
            name='gamma', lower=10 ** -5, upper=10 ** 3, default_value=10 **-5, log=True
        )        
        learning_rate = UniformFloatHyperparameter(
            name='learning_rate', lower=10 ** -5, upper=1, default_value=0.3, log=True
        )
        max_depth = UniformIntegerHyperparameter(
            name='max_depth', lower=1, upper=20, default_value=6, log=False
        )
        min_child_weight = UniformFloatHyperparameter(
            name='min_child_weight', lower=10 ** -5, upper=10 ** 3, default_value=1, log=True
        )
        n_estimators = UniformIntegerHyperparameter(
            name='n_estimators', lower=1, upper=10 ** 3, default_value=6, log=True
        )
        reg_alpha = UniformFloatHyperparameter(
            name='reg_alpha', lower=10 ** -5, upper=10 ** 3, default_value= 10 ** -5, log=True
        )
        reg_lambda = UniformFloatHyperparameter(
            name='reg_lambda', lower=10 ** -5, upper=10 ** 3, default_value=1, log=True
        )
        subsample = UniformFloatHyperparameter(
            name='subsample', lower=10 ** -5, upper=1, default_value=1, log=True
        )             

        cs.add_hyperparameters([colsample_bytree, gamma, learning_rate, max_depth, min_child_weight, n_estimators, reg_alpha, reg_lambda, subsample])
        return cs


# Add XGBoost component to auto-sklearn.
autosklearn.pipeline.components.regression.add_regressor(XGBoostRegression)
cs = XGBoostRegression.get_hyperparameter_search_space()
print(cs)


Configuration space object:
  Hyperparameters:
    colsample_bytree, Type: UniformFloat, Range: [1e-05, 1.0], Default: 1.0, on log-scale
    gamma, Type: UniformFloat, Range: [1e-05, 1000.0], Default: 1e-05, on log-scale
    learning_rate, Type: UniformFloat, Range: [1e-05, 1.0], Default: 0.3, on log-scale
    max_depth, Type: UniformInteger, Range: [1, 20], Default: 6
    min_child_weight, Type: UniformFloat, Range: [1e-05, 1000.0], Default: 1.0, on log-scale
    n_estimators, Type: UniformInteger, Range: [1, 1000], Default: 6, on log-scale
    reg_alpha, Type: UniformFloat, Range: [1e-05, 1000.0], Default: 1e-05, on log-scale
    reg_lambda, Type: UniformFloat, Range: [1e-05, 1000.0], Default: 1.0, on log-scale
    subsample, Type: UniformFloat, Range: [1e-05, 1.0], Default: 1.0, on log-scale



In [ ]:
# Second Step: Optimize Hyperparameter with automl.

reg = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=time_left_for_this_task,
    per_run_time_limit=per_run_time_limit,
    include={
        'regressor': ['XGBoostRegression'],
        'feature_preprocessor': ['no_preprocessing']
    },
    ensemble_size=1
)
reg.fit(X_train, y_train)

[WARNING] [2022-06-29 09:27:23,474:Client-AutoMLSMBO(1)::ae18311c-f78d-11ec-8204-0242ac1c0002] Configuration 197 not found
[WARNING] [2022-06-29 09:27:23,474:Client-AutoMLSMBO(1)::ae18311c-f78d-11ec-8204-0242ac1c0002] Configuration 235 not found
[WARNING] [2022-06-29 09:27:23,474:Client-AutoMLSMBO(1)::ae18311c-f78d-11ec-8204-0242ac1c0002] Configuration 270 not found
[WARNING] [2022-06-29 09:27:23,474:Client-AutoMLSMBO(1)::ae18311c-f78d-11ec-8204-0242ac1c0002] Configuration 163 not found
[WARNING] [2022-06-29 09:27:23,474:Client-AutoMLSMBO(1)::ae18311c-f78d-11ec-8204-0242ac1c0002] Configuration 67 not found
[WARNING] [2022-06-29 09:27:23,474:Client-AutoMLSMBO(1)::ae18311c-f78d-11ec-8204-0242ac1c0002] Configuration 34 not found
[WARNING] [2022-06-29 09:27:23,474:Client-AutoMLSMBO(1)::ae18311c-f78d-11ec-8204-0242ac1c0002] Configuration 86 not found
[WARNING] [2022-06-29 09:27:23,474:Client-AutoMLSMBO(1)::ae18311c-f78d-11ec-8204-0242ac1c0002] Configuration 240 not found
[WARNING] [2022-06-

AutoSklearnRegressor(ensemble_size=1,
                     include={'feature_preprocessor': ['no_preprocessing'],
                              'regressor': ['XGBoostRegression']},
                     per_run_time_limit=20, time_left_for_this_task=180)

In [ ]:
# Third Step: Save best params and best score.
model_dict = reg.show_models()
key = list(model_dict.keys())[0]
print(model_dict)

xgbParams = model_dict[key]['sklearn_regressor'].get_params()

{74: {'model_id': 74, 'rank': 1, 'cost': 0.12409828692463121, 'ensemble_weight': 1.0, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f76e4bb4050>, 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f76e5c65450>, 'regressor': <autosklearn.pipeline.components.regression.RegressorChoice object at 0x7f76e5c65110>, 'sklearn_regressor': XGBRegressor(colsample_bytree=0.7582507145307963, gamma=0.06806958191789281,
             learning_rate=0.1126227067000315, max_depth=2,
             min_child_weight=3.7170931233984515e-05, missing=nan,
             n_estimators=306, nthread=-1, random_state=1,
             reg_alpha=4.375827312211721e-05, reg_lambda=0.11288071370176124,
             subsample=0.4677215972307256)}}


# Optimize LGBMRegressor
Hyperparameters: num_leaves,
                              learning_rate, n_estimators,
                              max_bin, bagging_fraction,
                              bagging_freq, feature_fraction,
                              feature_fraction_seed, bagging_seed,
                              min_data_in_leaf, min_sum_hessian_in_leaf

NOT FINISHED. Documentation and provided link have different arguments in algorithm.
https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html
https://www.kaggle.com/code/eraaz1/a-comprehensive-guide-to-advanced-regression/notebook

In [ ]:
'''

# First Step: Extend autosklearn


from ConfigSpace.configuration_space import ConfigurationSpace
from ConfigSpace.hyperparameters import UniformFloatHyperparameter, \
    UniformIntegerHyperparameter, CategoricalHyperparameter
from ConfigSpace.conditions import EqualsCondition

import sklearn.metrics
import autosklearn.regression
import autosklearn.pipeline.components.regression
from autosklearn.pipeline.components.base import AutoSklearnRegressionAlgorithm
from autosklearn.pipeline.constants import SPARSE, DENSE, \
    SIGNED_DATA, UNSIGNED_DATA, PREDICTIONS

class LGBMRegressionExtendedAutoML(AutoSklearnRegressionAlgorithm):

    def __init__(self, num_leaves, learning_rate, n_estimators, max_bin, bagging_fraction, bagging_freq, feature_fraction, feature_fraction_seed, bagging_seed, min_data_in_leaf, min_sum_hessian_in_leaf, random_state=1):
        self.objective = 'regression'
        self.num_leaves = num_leaves
        self.learning_rate = learning_rate
        self.n_estimators = n_estimators
        self.max_bin = max_bin
        self.bagging_fraction = bagging_fraction
        self.bagging_freq = bagging_freq
        self.feature_fraction = feature_fraction
        self.feature_fraction_seed = feature_fraction_seed
        self.bagging_seed = bagging_seed
        self.min_data_in_leaf = min_data_in_leaf
        self.min_sum_hessian_in_leaf = min_sum_hessian_in_leaf
        self.random_state = random_state
        self.estimator = None

    def fit(self, X, y):
        from lightgbm import LGBMRegressor
        self.estimator = LGBMRegressor(
            objective=self.objective,
            num_leaves=self.num_leaves,
            learning_rate=self.learning_rate,
            n_estimators=self.n_estimators,
            max_bin = self.max_bin,
            bagging_fraction = self.bagging_fraction,
            bagging_freq = self.bagging_freq,
            feature_fraction = self.feature_fraction,
            feature_fraction_seed = self.feature_fraction_seed,
            bagging_seed = self.bagging_seed,
            min_data_in_leaf = self.min_data_in_leaf,
            min_sum_hessian_in_leaf = self.min_sum_hessian_in_leaf
        )
        self.estimator.fit(X, y)
        return self

    def predict(self, X):
        if self.estimator is None:
            raise NotImplementedError
        return self.estimator.predict(X)

    @staticmethod
    def get_properties(dataset_properties=None):
        return {
            'shortname': 'LGBM',
            'name': 'LGBMRegressor',
            'handles_regression': True,
            'handles_classification': False,
            'handles_multiclass': False,
            'handles_multilabel': False,
            'handles_multioutput': True,
            'is_deterministic': True,
            'input': (SPARSE, DENSE, UNSIGNED_DATA, SIGNED_DATA),
            'output': (PREDICTIONS,)
        }

    @staticmethod
    def get_hyperparameter_search_space(dataset_properties=None):
        cs = ConfigurationSpace()
        learning_rate = UniformFloatHyperparameter(
            name='learning_rate', lower=10 ** -5, upper=1, default_value=0.3, log=True
        )
        n_estimators = UniformIntegerHyperparameter(
            name='n_estimators', lower=1, upper=10 ** 3, default_value=6, log=True
        )
        num_leaves = UniformIntegerHyperparameter(
            name='num_leaves', lower=1, upper=10 ** 3, default_value=6, log=True
        )

        cs.add_hyperparameters([alpha, kernel, gamma, degree, coef0])
        return cs


# Add KRR component to auto-sklearn.
autosklearn.pipeline.components.regression.add_regressor(KernelRidgeRegression)
cs = KernelRidgeRegression.get_hyperparameter_search_space()
print(cs)
'''

Configuration space object:
  Hyperparameters:
    alpha, Type: UniformFloat, Range: [1e-05, 1.0], Default: 1.0, on log-scale
    coef0, Type: UniformFloat, Range: [0.01, 100.0], Default: 1.0, on log-scale
    degree, Type: UniformInteger, Range: [2, 5], Default: 3
    gamma, Type: UniformFloat, Range: [1e-05, 1.0], Default: 0.1, on log-scale
    kernel, Type: Categorical, Choices: {polynomial, rbf}, Default: polynomial
  Conditions:
    coef0 | kernel == 'polynomial'
    degree | kernel == 'polynomial'



# Optimize GPR

In [ ]:
reg = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=time_left_for_this_task,
    per_run_time_limit=per_run_time_limit,
    include={
        'regressor': ['gaussian_process'],
        'feature_preprocessor': ['no_preprocessing']
    },
    ensemble_size=1
)
reg.fit(X_train, y_train)

[WARNING] [2022-06-29 09:30:19,396:Client-AutoMLSMBO(1)::16ed27a6-f78e-11ec-8204-0242ac1c0002] Configuration 197 not found
[WARNING] [2022-06-29 09:30:19,397:Client-AutoMLSMBO(1)::16ed27a6-f78e-11ec-8204-0242ac1c0002] Configuration 235 not found
[WARNING] [2022-06-29 09:30:19,397:Client-AutoMLSMBO(1)::16ed27a6-f78e-11ec-8204-0242ac1c0002] Configuration 270 not found
[WARNING] [2022-06-29 09:30:19,397:Client-AutoMLSMBO(1)::16ed27a6-f78e-11ec-8204-0242ac1c0002] Configuration 163 not found
[WARNING] [2022-06-29 09:30:19,397:Client-AutoMLSMBO(1)::16ed27a6-f78e-11ec-8204-0242ac1c0002] Configuration 67 not found
[WARNING] [2022-06-29 09:30:19,397:Client-AutoMLSMBO(1)::16ed27a6-f78e-11ec-8204-0242ac1c0002] Configuration 34 not found
[WARNING] [2022-06-29 09:30:19,397:Client-AutoMLSMBO(1)::16ed27a6-f78e-11ec-8204-0242ac1c0002] Configuration 86 not found
[WARNING] [2022-06-29 09:30:19,397:Client-AutoMLSMBO(1)::16ed27a6-f78e-11ec-8204-0242ac1c0002] Configuration 240 not found
[WARNING] [2022-06-

AutoSklearnRegressor(ensemble_size=1,
                     include={'feature_preprocessor': ['no_preprocessing'],
                              'regressor': ['gaussian_process']},
                     per_run_time_limit=20, time_left_for_this_task=180)

In [ ]:
# Third Step: Save best params and best score.
model_dict = reg.show_models()
key = list(model_dict.keys())[0]
print(model_dict)
gpr_best_params = [model_dict[key]['sklearn_regressor'].alpha, model_dict[key]['sklearn_regressor'].kernel]
gpr_best_score = model_dict[key]['cost']

print('Best Parameter for alpha: {}. Best Parameter for Kernel: {}. Best Score: {}'.format(gpr_best_params[0], gpr_best_params[1], gpr_best_score))
gprParams = model_dict[key]['sklearn_regressor'].get_params()

{100: {'model_id': 100, 'rank': 1, 'cost': 0.0038797102193601773, 'ensemble_weight': 1.0, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f76e4ebb1d0>, 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f76e5c13a90>, 'regressor': <autosklearn.pipeline.components.regression.RegressorChoice object at 0x7f76e5c13910>, 'sklearn_regressor': GaussianProcessRegressor(alpha=9.33680095345871e-05,
                         kernel=RBF(length_scale=[1, 1, 1, 1, 1, 1, 1, 1, 1]),
                         n_restarts_optimizer=10, normalize_y=True,
                         random_state=1)}}
Best Parameter for alpha: 9.33680095345871e-05. Best Parameter for Kernel: RBF(length_scale=[1, 1, 1, 1, 1, 1, 1, 1, 1]). Best Score: 0.0038797102193601773


# Optimize MLP

In [ ]:
reg = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=time_left_for_this_task,
    per_run_time_limit=per_run_time_limit,
    include={
        'regressor': ['mlp'],
        'feature_preprocessor': ['no_preprocessing']
    },
    ensemble_size=1
)
reg.fit(X_train, y_train)

[WARNING] [2022-06-29 09:33:13,083:Client-AutoMLSMBO(1)::7e90aac6-f78e-11ec-8204-0242ac1c0002] Configuration 197 not found
[WARNING] [2022-06-29 09:33:13,083:Client-AutoMLSMBO(1)::7e90aac6-f78e-11ec-8204-0242ac1c0002] Configuration 235 not found
[WARNING] [2022-06-29 09:33:13,083:Client-AutoMLSMBO(1)::7e90aac6-f78e-11ec-8204-0242ac1c0002] Configuration 270 not found
[WARNING] [2022-06-29 09:33:13,083:Client-AutoMLSMBO(1)::7e90aac6-f78e-11ec-8204-0242ac1c0002] Configuration 163 not found
[WARNING] [2022-06-29 09:33:13,083:Client-AutoMLSMBO(1)::7e90aac6-f78e-11ec-8204-0242ac1c0002] Configuration 67 not found
[WARNING] [2022-06-29 09:33:13,083:Client-AutoMLSMBO(1)::7e90aac6-f78e-11ec-8204-0242ac1c0002] Configuration 34 not found
[WARNING] [2022-06-29 09:33:13,083:Client-AutoMLSMBO(1)::7e90aac6-f78e-11ec-8204-0242ac1c0002] Configuration 86 not found
[WARNING] [2022-06-29 09:33:13,083:Client-AutoMLSMBO(1)::7e90aac6-f78e-11ec-8204-0242ac1c0002] Configuration 240 not found
[WARNING] [2022-06-

AutoSklearnRegressor(ensemble_size=1,
                     include={'feature_preprocessor': ['no_preprocessing'],
                              'regressor': ['mlp']},
                     per_run_time_limit=20, time_left_for_this_task=180)

In [ ]:
# Third Step: Save best params and best score.
model_dict = reg.show_models()
key = list(model_dict.keys())[0]
print(model_dict)
mlpParams = model_dict[key]['sklearn_regressor'].get_params()

{68: {'model_id': 68, 'rank': 1, 'cost': 0.0114334667982533, 'ensemble_weight': 1.0, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f76e5c61e90>, 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f76e5c44810>, 'regressor': <autosklearn.pipeline.components.regression.RegressorChoice object at 0x7f76e5c44410>, 'sklearn_regressor': MLPRegressor(alpha=0.09780071940021463, beta_1=0.999, beta_2=0.9,
             hidden_layer_sizes=(100, 100),
             learning_rate_init=0.003338891034046557, max_iter=256,
             n_iter_no_change=32, random_state=1, validation_fraction=0.0,
             verbose=0, warm_start=True)}}


# Retrain and Predict with optimized Hyperparameters from AutoML.

In [ ]:
'''Initialize all the regression models object we are interested in.'''
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

In [ ]:
linearOpt = LinearRegression(n_jobs = -1)
lassoOpt = Lasso().set_params(**lassoParams)
ridgeOpt = Ridge().set_params(**ridgeParams)
krOpt = KernelRidge().set_params(**krrParams)
elntOpt = ElasticNet().set_params(**enParams)
dtOpt = DecisionTreeRegressor().set_params(**dtParams)
svmOpt = SVR().set_params(**svrParams)
knnOpt = KNeighborsRegressor().set_params(**knnParams)
rfOpt =  RandomForestRegressor().set_params(**rfParams)
etOpt = ExtraTreesRegressor().set_params(**etParams)
abOpt = AdaBoostRegressor().set_params(**abParams)
gbOpt = GradientBoostingRegressor().set_params(**gbParams)
xgbOpt = XGBRegressor().set_params(**xgbParams)
gprOpt = GaussianProcessRegressor().set_params(**gprParams)
mlpOpt = MLPRegressor().set_params(**mlpParams)

In [ ]:
'''Training accuracy of our regression models. By default score method returns coefficient of determination (r_squared).'''
def train_r2(model):
    model.fit(X_train, y_train)
    return model.score(X_train, y_train)

'''Calculate and plot the training accuracy.'''
models = [linearOpt, lassoOpt, ridgeOpt, krOpt, elntOpt, dtOpt, svmOpt, knnOpt, rfOpt, etOpt, abOpt, gbOpt, xgbOpt, gprOpt, mlpOpt]
training_score = []
for model in models:
    training_score.append(train_r2(model))
    
'''Plot dataframe of training accuracy.'''
train_score = pd.DataFrame(data = training_score, columns = ['Training_R2'])
train_score.index = ['LR', 'LSO', 'RIDGE', 'KR', 'ELNT', 'DT', 'SVM', 'KNN', 'RF', 'ET', 'AB', 'GB', 'XGB', 'GPR', 'MLP']
train_score = (train_score*100).round(4)
scatter_plot(train_score.index, train_score['Training_R2'], 'Training Score (R_Squared)', 'Models','% Training Score', 30, 'Rainbow')


import os  

os.makedirs('/content/drive/MyDrive/Colab Notebooks/AutoML', exist_ok=True)  

train_score.to_csv('/content/drive/MyDrive/Colab Notebooks/AutoML/train_score_AutoMLParams.csv')  

[10:12:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
train_test_split_rmse = []
for model in models:
    train_test_split_rmse.append(train_test_split_score(model))

'''Plot data frame of train test rmse'''
train_test_score = pd.DataFrame(data = train_test_split_rmse, columns = ['Train_Test_RMSE'])
train_test_score.index = ['LR', 'LSO', 'RIDGE', 'KR', 'ELNT', 'DT', 'SVM', 'KNN', 'RF', 'ET', 'AB', 'GB', 'XGB', 'GPR', 'MLP']
train_test_score = train_test_score.round(5)
x = train_test_score.index
y = train_test_score['Train_Test_RMSE']
title = "Models' Test Score (RMSE) on Holdout(10%) Set"
scatter_plot(x, y, title, 'Models','RMSE', 30, 'RdBu')


import os  

os.makedirs('/content/drive/MyDrive/Colab Notebooks/AutoML', exist_ok=True)  

train_score.to_csv('/content/drive/MyDrive/Colab Notebooks/AutoML/train__test_score_AutoMLParams.csv')  

[10:13:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
'''Function to compute cross validation scores.'''
def cross_validate(model):
    from sklearn.model_selection import cross_val_score
    neg_x_val_score = cross_val_score(model, X_train, y_train, cv = 10, n_jobs = -1, scoring = 'neg_mean_squared_error')
    x_val_score = np.round(np.sqrt(-1*neg_x_val_score), 5)
    return x_val_score.mean()

models = [linear, lassoOpt, ridgeOpt, krOpt, elntOpt, dtOpt, svmOpt, knnOpt, rfOpt, etOpt, abOpt, gbOpt, xgbOpt, gprOpt, mlpOpt]
'''Calculate cross validation score of differnt models and plot them.'''
cross_val_scores = []
for model in models:
    cross_val_scores.append(cross_validate(model))

'''Plot data frame of cross validation scores.'''
x_val_score = pd.DataFrame(data = cross_val_scores, columns = ['Cross Validation Scores (RMSE)'])
x_val_score.index = ['LR', 'LSO', 'RIDGE', 'KR', 'ELNT', 'DT', 'SVM', 'KNN', 'RF', 'ET', 'AB', 'GB', 'XGB', 'GPR', 'MLP']
x_val_score = x_val_score.round(5)
x = x_val_score.index
y = x_val_score['Cross Validation Scores (RMSE)']
title = "Models' 10-fold Cross Validation Scores (RMSE)"
scatter_plot(x, y, title, 'Models','RMSE', 30, 'Viridis')


import os  

os.makedirs('/content/drive/MyDrive/Colab Notebooks/AutoML', exist_ok=True)  

train_score.to_csv('/content/drive/MyDrive/Colab Notebooks/AutoML/10fCV_AutoMLParams.csv')  

# Feature Importance

In [ ]:
'''Function to plot scatter plot of feature importance of lasso, ridge and elastic net.'''
def plot_feature_importance(model, title):
    model.fit(X_train, y_train)
    coef = pd.DataFrame({'Feature':data_cols,'Importance':np.transpose(model.coef_)})
    coef = coef.set_index('Feature')
    trace = go.Scatter(x = coef.index, y = coef.Importance, mode = 'markers',
            marker = dict(color = np.random.randn(500), size = 20, showscale = True, colorscale = 'Hot'))
    layout = go.Layout(hovermode = 'closest', title = title, yaxis = dict(title = 'Importance'))
    fig = go.Figure(data = [trace], layout = layout)
    iplot(fig)

'''Now plot feature importance of optimized lasso, ridge, and elastic net.'''
plot_feature_importance(lassoOpt, 'Lasso Feature Importance')
plot_feature_importance(ridgeOpt, 'Ridge Feature Importance')
plot_feature_importance(elntOpt, 'ElasticNet Feature Importance')

